In [5]:
from experta import *
from schema import *
from experta.utils import *
from functools import reduce, singledispatch

In [7]:
''' helper constant '''


class B_COLORS:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


CF_STRINGS = {
    'Yes': 1.0,
    'No': 0.0
}


class DiseaseStates:
    INITIAL = 'INITIAL'
    SY_CF_INPUTTED = 'SY_CF_INPUTTED'
    DIAGNOSED = 'DIAGNOSED'

    values = [
        'INITIAL',
        'SY_CF_INPUTTED',
        'DIAGNOSED'
    ]

In [9]:
''' data table '''
# P1:
#   D1:
#       s1: 0.3
#       s2: 0.2
#   D2:
#       s1: 0.5
#       s3: 0.5
#
# P2
#   D1:
#       s1: 0.2
#       s2: 0.3
#       s4: 0.6
#   D3:
#       s3: 0.9
#       s4: 0.1


Disease_Data = {
   "ملفوف - Cabbage": [
      {
         "name": "نيماتودا تعقد الجذور\nRoot Knot\n",
         "link": "https://www.al-hakem.com/نيماتودا-تعقد-الجذور-على-الملفوف-و-الق/",
         "details": "العامل المسبب : النيماتودا Meloidogyne spp\nتستطيع هذه الديدان الخيطية البقاء على قيد الحياة في بقايا الجذر المصابة. تنجذب الأحياء الصغيرة إلى جذور النباتات المضيفة وتتغذى على أنسجة الجذر. يمكن أن يحدث الضرر الأخطر في التربة الرملية ذات الرطوبة المعتدلة",
         "symptoms": [
            {
               "name": "Abundant root and smell over the point of infection",
               "nameAr": " تكاثر جذور غزير ورائحة فوق نقطة العدوى",
               "CF": 0.9939513612514557
            },
            {
               "name": "Chlorosis in the leaves",
               "nameAr": "تبرقش على الأوراق",
               "CF": 0.30824366763592626
            },
            {
               "name": "the crop is small",
               "nameAr": " المحصول الناتج يكون صغيرا بشكل عام",
               "CF": 0.10713270430420974
            }
         ]
      },
      {
         "name": "موزاييك الفجل \nRadish Mosaic\n",
         "link": "https://www.al-hakem.com/موزاييك-الفجل-على-الملفوف-و-القرنبيط/",
         "details": "العامل المسبب فيروس : (Radish mosaic virus (RaMV\nينتقل فيروس موزاييك الفجل RaMV عن طريق الخنافس المختلفة. الفيروس موجود في كل من نباتات المحاصيل والأعشاب الضارة ، والتي تعمل بمثابة خزانات لانتشار المرض.",
         "symptoms": [
            {
               "name": "Chlorosis in the leaves",
               "nameAr": "تبرقش على الأوراق",
               "CF": 0.8590444672117369
            },
            {
               "name": "Annular spots and leaf deformation",
               "nameAr": " بقع حلقية وتشوه الأوراق",
               "CF": 0.35287527731010604
            },
            {
               "name": "Necrosis in the leaves",
               "nameAr": " نخر منتظم في الأوراق",
               "CF": 0.045855530255609955
            }
         ]
      },
      {
         "name": "نقص البورون \n(Boron (B\n",
         "link": "https://www.al-hakem.com/نقص-البورون-على-الملفوف-و-القرنبيط/",
         "details": "",
         "symptoms": [
            {
               "name": "change in the color ",
               "nameAr": " تغير اللون الداخلي للنبات",
               "CF": 0.16503960185079503
            },
            {
               "name": "Leg otter crumbling",
               "nameAr": " تفتت أوتكسير الساق عند قاعدة الرأس",
               "CF": 0.9175340093464005
            },
            {
               "name": "Cork-like roughness",
               "nameAr": "خشونة على شكل شبيه بالفلين ، على السطح السفلي من منتصف الضلع للأوراق",
               "CF": 0.6
            },
            {
               "name": "reddish brown color inside and out",
               "nameAr": " اسمرار (بني محمر) من الداخل و الخارج",
               "CF": 0.4415518637712319
            }
         ]
      },
      {
         "name": "نقص االفوسفور \n(Phosphorus (P\n",
         "link": "https://www.al-hakem.com/نقص-االفوسفور-على-الملفوف-و-القرنبيط/",
         "details": "",
         "symptoms": [
            {
               "name": "slow growth",
               "nameAr": "النمو ببطء",
               "CF": 0.35969603296219155
            },
            {
               "name": "The leg is thin, short and fragile",
               "nameAr": " السوق تكون رقيقة وقصيرة وهشة",
               "CF": 0.26860082733157975
            },
            {
               "name": "The leaves are purple",
               "nameAr": " يظهر على الأوراق لونًا أرجوانيًا",
               "CF": 0.7799719195996083
            }
         ]
      },
      {
         "name": "نقص النيتروجين \nNitrogen Deficiency in cabbage\n",
         "link": "https://www.al-hakem.com/نقص-النيتروجين-على-الملفوف-و-القرنبيط/",
         "details": "",
         "symptoms": [
            {
               "name": "smaller leaves",
               "nameAr": "أوراق أصغر من الأوراق الطبيعية",
               "CF": 0.013501704603971154
            },
            {
               "name": "yellowish-green leaves",
               "nameAr": "أوراق لونها أخضر باهت مصفر",
               "CF": 0.8394240572321786
            },
            {
               "name": "The leg is thin and solid",
               "nameAr": "الساق تكون رقيقة وصلبة",
               "CF": 0.5187254942892808
            },
            {
               "name": "slow growth",
               "nameAr": "النمو ببطء",
               "CF": 0.3645759734530869
            }
         ]
      },
      {
         "name": "نقص المولبيديوم\n(Molybdenum (Mo",
         "link": "https://www.al-hakem.com/نقص-المولبيديوم-على-الملفوف-و-القرنبي/",
         "details": "محاصيل الملفوف حساسة جدا لنقص الموليبديوم ، ويظهر على الزهرة أو القرنبيط باعتباره مؤشر جيد جدا لهذا النقص.",
         "symptoms": [
            {
               "name": "yellowish-green leaves",
               "nameAr": "أوراق لونها أخضر باهت مصفر",
               "CF": 0.9364039681752738
            },
            {
               "name": "Distorted narrow leaves",
               "nameAr": "أوراق ضيقة مشوهة",
               "CF": 0.8267433943948889
            },
            {
               "name": "Perforated leaves",
               "nameAr": " أوراق مفتوحة",
               "CF": 0.3809011842030361
            },
            {
               "name": "Leaf edges eroded",
               "nameAr": "أطراف أوراق متآكلة",
               "CF": 0.07421895116574251
            },
            {
               "name": "smaller leaves",
               "nameAr": "أوراق أصغر من الأوراق الطبيعية",
               "CF": 0.7109750830095546
            }
         ]
      },
      {
         "name": "نقص المغنيسيوم \n(Magnesium (Mg\n",
         "link": "https://www.al-hakem.com/نقص-المغنيسيوم-على-الملفوف-و-القرنبيط/",
         "details": "يحدث نقص المغنيسيوم أساسًا في التربة الحمضية ، وفي التربة الخفيفة جدًا التي تخضع للغسيل أو في التربة ذات مستوى البوتاسيوم المرتفع جدًا.",
         "symptoms": [
            {
               "name": "The leaves are stained violet red",
               "nameAr": "تلون الأوراق باللون الأحمر البنفسجي",
               "CF": 0.15601163460921186
            },
            {
               "name": "Chlorosis in the leaves",
               "nameAr": "تبرقش على الأوراق",
               "CF": 0.44555818384733037
            },
            {
               "name": "leaves fall",
               "nameAr": "تساقط الاوراق",
               "CF": 0.7077336690573596
            }
         ]
      },
      {
         "name": "نقص البوتاسيوم \n(Potassium (K\n",
         "link": "https://www.al-hakem.com/نقص-البوتاسيوم-على-الملفوف-و-القرنبيط/",
         "details": "تحتاج المحاصيل الملفوفية إلى كميات عالية من البوتاسيوم. تشير تحاليل النبات إلى أنها تستهلك كمية من البوتاسيوم تقريباً مثل النيتروجين. على الرغم من أن الناتج الإجمالي نادرا ما يتأثر.",
         "symptoms": [
            {
               "name": "Heads tend to be softer and softer",
               "nameAr": " الرؤوس تميل إلى أن تكون أكثر نعومة ورخوة",
               "CF": 0.8471882973087432
            },
            {
               "name": "Leaf edges soft and chlorosis",
               "nameAr": "هوامش وأطراف الأوراق تصبح رخوة (ذابلة) وتصاب بالبرقشة ",
               "CF": 0.9918179679100653
            },
            {
               "name": "Leaves broken",
               "nameAr": " تكسير الأوراق",
               "CF": 0.13944736458471652
            }
         ]
      },
      {
         "name": "نقص الكاسيوم (Calcium (Ca\n",
         "link": "https://www.al-hakem.com/نقص-الكاسيوم-على-الملفوف-و-القرنبيط/",
         "details": "",
         "symptoms": [
            {
               "name": "Leaf edges pale, slim and thin",
               "nameAr": "أطراف أوراق وهوامش شاحبة ونحيفة و رقيقة",
               "CF": 0.7084217726302577
            },
            {
               "name": "leaves die",
               "nameAr": "تموت الأوراق في النهاية",
               "CF": 0.5241561168811746
            }
         ]
      }
   ],
   "زيتون - olive": [
      {
         "name": "نقص الفوسفور Phosphorus deficiency\n",
         "link": "https://www.al-hakem.com/نقص-الفوسفور-على-الزيتون/",
         "details": "",
         "symptoms": [
            {
               "name": "Leaf edges soft",
               "nameAr": "ذبول ورخاوة حواف الأوراق وتموجها",
               "CF": 0.9332480726984942
            },
            {
               "name": "small bracnhes",
               "nameAr": "فروع أصغر من المعتاد",
               "CF": 0.2
            },
            {
               "name": "decreased in the crop",
               "nameAr": "قلة ونقص في كمية المحصول الناتج",
               "CF": 0.5
            },
            {
               "name": "small fruits",
               "nameAr": "ثمار الزيتون صغيرة الحجم",
               "CF": 0.2
            },
            {
               "name": "small leaves",
               "nameAr": "أوراق صغيرة متقزمة",
               "CF": 0.2
            }
         ]
      },
      {
         "name": "مرض بقع أوراق الزيتون CERCOSPORA CLADOSPORIOIDES\n",
         "link": "https://www.al-hakem.com/مرض-بقع-أوراق-الزيتون/",
         "details": "",
         "symptoms": [
            {
               "name": "dark gray color on leaves",
               "nameAr": "تلون أسمر رصاصي أو رمادي غامق على السطح العلوي للأوراق",
               "CF": 0.8198425668571234
            },
            {
               "name": "leaves fall",
               "nameAr": "تساقط الأوراق",
               "CF": 0.5575576633650063
            },
            {
               "name": "Leaves turn brown on the upper surface and gray on the lower surface",
               "nameAr": "تتحول الأوراق المتساقطة قبل الأوان إلى اللون البني على السطح العلوي والى الرمادي على السطح السفلي",
               "CF": 0.7451581305189018
            },
            {
               "name": "brown fruits",
               "nameAr": "ثمار الزيتون تحوي بقع  باللون البني المحمر",
               "CF": 0.2
            }
         ]
      },
      {
         "name": "آفة عين الطاووس Peacock eye spots\n",
         "link": "https://www.al-hakem.com/آفة-عين-الطاؤوس-على-الزيتون/",
         "details": "مرض فطري يعد من أخطر الأمراض التي تصيب الزيتون في البلاد العربية . هذا الفطر يصيب أوراق الزيتون, عند الاصابة القوية تتساقط الاوراق.و البساتين التي تكثر فيها الرطوبة يكون خطر الاصابة أعلى.",
         "symptoms": [
            {
               "name": "leaves fall",
               "nameAr": "تساقط الأوراق",
               "CF": 0.7176997129939994
            },
            {
               "name": "Black gray circular spots, on leaves",
               "nameAr": "ظهور بقع دائرية لونها أسود رمادي خاصة على السطح العلوي للورقة",
               "CF": 0.6640094619309467
            },
            {
               "name": "spots like the peacock eye",
               "nameAr": "ظهور دوائر صفراء حول البقع السوداء تشبة عين الطاووس",
               "CF": 0.8
            },
            {
               "name": "leaves yellowish ",
               "nameAr": "اصفرار الأوراق",
               "CF": 0.8538536986206116
            },
            {
               "name": "stop growth the branches",
               "nameAr": "توقف نمو الأغصان",
               "CF": 0.2953014714215606
            }
         ]
      },
      {
         "name": "النيماتودا\n(root knot nematode)\n",
         "link": "https://www.al-hakem.com/النيماتودا-على-الزيتون/",
         "details": "العامل المسبب : Meloidogyne Spp\nيعتبرها الكثير من علماء أمراض النبات أحد أهم وأخطر خمسة مسببات مرضية اقتصادية في العالم ويعود ذلك إلى عدة عوامل من أهمها انتشارها السريع والواسع في جميع أنحاء العالم ومداها العائلي الواسع وكذلك تعاونها مع الأحياء الأخرى وخاصة الفطريات والبكتريا في إحداث الكثير من الأمراض النباتية المركبة\nتصل الخسارة نتيجة الإصابة بهذة الآفة إلى فقدان كامل للمحصول وفى حالة الإصابة الخفيفة فإن الخسارة لاتقل عن 25%",
         "symptoms": [
            {
               "name": "Leaves dry",
               "nameAr": "ذبول و جفاف الاوراق و موتها",
               "CF": 0.8161907632115631
            },
            {
               "name": "decreased in the crop",
               "nameAr": "قلة ونقص في كمية المحصول الناتج",
               "CF": 0.20562528658174306
            },
            {
               "name": "spots or tumers on the roots",
               "nameAr": "وجود عقد أو أورام على جذور النباتات المصابة",
               "CF": 0.804636536337743
            },
            {
               "name": "nodes inside the texture which pear-shaped females",
               "nameAr": "وجود عقد بداخلها الإناث الكمثرية الشكل والتي تكون مدفونة في الأنسجة",
               "CF": 0.25960595906497497
            },
            {
               "name": "Larvae inside the roots",
               "nameAr": "وجود أكياس البيض على سطح العقد والإناث واليرقات داخل الجذور",
               "CF": 0.5146542241607056
            },
            {
               "name": "The roots become many nodes ",
               "nameAr": "تتحول الجذور إلى عقد كثيرة مع وجود تحلل لأنسجة القشرة",
               "CF": 0.3
            },
            {
               "name": "The roots hold root knots on the newly developed roots",
               "nameAr": "تعقد الجذور عقدا جذرية على الجذور الحديثة النمو",
               "CF": 0.9143060308410758
            }
         ]
      },
      {
         "name": "ذبابة الزيتون Olive fly \n",
         "link": "https://www.al-hakem.com/ذبابة-ثمار-الزيتون/",
         "details": "الاسم العلمي  : Dacus oleae\nهي ذبابة متوسطة الحجم يصل طولها نحو 5مم. الجناح شفاف وعلى طرفه الخارجي الأمامي بقعة صغيرة مصفرة اللون، و لون الصدر أصفر سمني و عليه خطان طوليان داكنان متقاطعان. الأرجل مصفرة، و لون البطن بني و على جانبيها أشرطة زرقاء داكنة",
         "symptoms": [
            {
               "name": "fruits have gray or black color",
               "nameAr": "تغير لون الثمرة إلى الرمادي المسود أو الأسود",
               "CF": 0.01596975650792054
            },
            {
               "name": "Sponge fruit",
               "nameAr": "ثمرة قوامها إسفنجي",
               "CF": 0.21004785636463152
            },
            {
               "name": "fall fruit",
               "nameAr": "تساقط الثمار",
               "CF": 0.789419848920949
            },
            {
               "name": "low quality oil",
               "nameAr": "جودة الزيت منخفض وتزيد حموضته",
               "CF": 0.5113337163684815
            }
         ]
      },
      {
         "name": "نقص النتروجين Nitrogen deficiency\n",
         "link": "https://www.al-hakem.com/نقص-النتروجين-على-الزيتون/",
         "details": "في بداية الموسم يستدل على أعراض نقص النيتروجين. مع ملاحظة أن زيادة أو قلة الرطوبة في التربة تحدث أعراضاً شبيهة بأعراض نقص النيتروجين.",
         "symptoms": [
            {
               "name": "slow growth the branches",
               "nameAr": "بطء نمو الأفرع",
               "CF": 0.3794335075613605
            },
            {
               "name": "leaves yellowish ",
               "nameAr": "اصفرار الأوراق",
               "CF": 0.9519409371972298
            },
            {
               "name": "leaves fall",
               "nameAr": "تساقط الأوراق",
               "CF": 0.06609484602634208
            },
            {
               "name": "small fruits",
               "nameAr": "ثمار الزيتون صغيرة الحجم",
               "CF": 0.3
            }
         ]
      },
      {
         "name": "مرض سل الزيتون  OLIVE KNOT\n",
         "link": "https://www.al-hakem.com/مرض-سل-الزيتون/",
         "details": "العامل المسبب : بكتريا   Pseudomonas syringae\nينتشر في المناطق الرطبة وشبه الرطبة و تعد الثآليل التي تتواجد على أفرع الزيتون أهم مصدر من مصادر العدوى\nتشاهد هذه الظاهرة في السنوات التي يكون فيها الربيع بارداً",
         "symptoms": [
            {
               "name": "Tumors on plant",
               "nameAr": "أورام أو ثآليل بحجم بين رأس الدبوس حتى حجم ثمرة الجوز تظهر على جميع أجزاء النبات المختلفة",
               "CF": 0.6
            },
            {
               "name": "Tumors appear at leaves ",
               "nameAr": "ظهور أورام على الأوراق",
               "CF": 0.8709651805298363
            },
            {
               "name": "Small spots on branches, become Large size drills on branches with dark surfaces",
               "nameAr": "ظهور نموات صغيرة ناعمة ذات لون أخضر في منطقة الإصابة على الأفرع والأغصان\nمع الوقت تصبح تدرنات ذات حجم كبير على الأفرع والأغصات ذات سطوح خارجية دكنة اللون ومتشققة",
               "CF": 0.2570823302249956
            },
            {
               "name": "circular brown spots on the fruit",
               "nameAr": "يظهر على سطح الثمرة المصابة بقع بنية دائرية اللون, والتي تتحول فيما بعد إلى اللون الغامق أو المائل للسواد وتصبح غائرة",
               "CF": 0.9007268160345544
            },
            {
               "name": "leaves fall",
               "nameAr": "تساقط الأوراق",
               "CF": 0.19742969112251296
            },
            {
               "name": "small fruits",
               "nameAr": "ثمار الزيتون صغيرة الحجم",
               "CF": 0.18369935727574715
            },
            {
               "name": "Dry branches",
               "nameAr": "ذبول و جفاف الأفرع و موتها",
               "CF": 0.052346161208547626
            },
            {
               "name": "weakness in plant growth",
               "nameAr": "ضعف عام في نمو الشجرة والنبات",
               "CF": 0.8004151398671137
            },
            {
               "name": "fruits have an undesirable smell",
               "nameAr": "الثمار المقطوفة يكون لها رائحة غير مرغوبة",
               "CF": 0.3548064629103922
            },
            {
               "name": "decreased in the crop",
               "nameAr": "قلة ونقص في كمية المحصول الناتج",
               "CF": 0.41382649437059504
            }
         ]
      },
      {
         "name": "ذبول الفيرتسيليوم على الزيتون Verticillium dahliae\n",
         "link": "https://www.al-hakem.com/ذبول-الفيرتسيليوم-على-الزيتون/",
         "details": "وهو من أهم الأمراض على الزيتون والذي يحدد وجوده امكانية زراعة الزيتون , وهو مرض فطري ينتقل عبر الجذور ويسبب ذبول و جفاف الاوراق الأفرع و موتها . يهاجم هذا الفطر الأشجار القوية أكثرمن الضعيفة. وقد يؤدي هذا المرض إلى موت الشجرة .",
         "symptoms": [
            {
               "name": "branches barren except at the top ",
               "nameAr": "تصبح أفرع الشجرة جرداء ما عدا بعض الأوراق في قمة الفرع العلوية",
               "CF": 0.8
            },
            {
               "name": "Leaves dry",
               "nameAr": "ذبول و جفاف الاوراق و موتها",
               "CF": 0.4231627021413239
            },
            {
               "name": "Dry branches",
               "nameAr": "ذبول و جفاف الأفرع و موتها",
               "CF": 0.8342965830639565
            },
            {
               "name": "Tree death",
               "nameAr": "موت الشجرة",
               "CF": 0.5762099897650267
            },
            {
               "name": "leaves yellowish ",
               "nameAr": "اصفرار الأوراق",
               "CF": 0.4
            },
            {
               "name": "Yellow section of the branch",
               "nameAr": "عند عمل مقطع في الأفرع نشاهد تلون المائل للأصفر",
               "CF": 0.2842181414839775
            }
         ]
      },
      {
         "name": "\nالتدرن التاجي على الزيتون \nCrown gall",
         "link": "https://www.al-hakem.com/التدرن-التاجي-على-الزيتون/",
         "details": "مسبب الافة :بكتريا Agrobacterium tumefaciens\nيصيب هذا المرض حوالي ستين عائلة نباتية تضم بالإضافة إلى اللوزيات أشجار الفاكهة مثل التفاح والكمثري والبرقوق والزيتون ونباتات الزينة مثل البوانسيانا",
         "symptoms": [
            {
               "name": "Tumors near the soil turn black",
               "nameAr": "تورمات في منطقة التاج  بالقرب من سطح التربة تتحول ألى اللون الأسود",
               "CF": 0.5759419888621393
            },
            {
               "name": "plants are meager",
               "nameAr": "النباتات المصابة هزيلة",
               "CF": 0.3059574120893026
            },
            {
               "name": "leaves yellowish ",
               "nameAr": "اصفرار الأوراق",
               "CF": 0.061873476507394276
            },
            {
               "name": "small leaves",
               "nameAr": "أوراق صغيرة متقزمة",
               "CF": 0.09361091607467165
            },
            {
               "name": "Tree death",
               "nameAr": "موت الشجرة",
               "CF": 0.5407990830141167
            },
            {
               "name": "Swollen leg and root tissue",
               "nameAr": "انتفاخ أنسجة منطقة الساق والجذر - عادةً ما يكون مكان الانتفاخ في نفس مكان التي توجد فيه الجروح وتكون عادة في المناطق التاجية على الساق فوق سطح التربة مباشرةً وفي بعض الأحيان تكون تحت سطح التربة أو على الأغصان .",
               "CF": 0.712460312376998
            },
            {
               "name": "smaller plants",
               "nameAr": "تقزم النبات",
               "CF": 0.44281247659787515
            }
         ]
      },
      {
         "name": "البثور أو الانثراكنوز على الزيتون\n OLIVE ANTHRACNOSE\n",
         "link": "https://www.al-hakem.com/البثور-الانثراكنوز-على-الزيتون/",
         "details": "العامل المسبب :الفطر Colletotrichum acutatum and C. gloeosporioides\nهذا المرض منتشر بشكل واسع في العديد من البلدان العربية ,و يؤثر بشكل عام على ثمار الزيتون، مما يسبب في فقدان الوزن 40-50%، وانخفاض أعداد الفاكهة , وانخفاض كمية الزيت .",
         "symptoms": [
            {
               "name": "fruits are embalmed and rot",
               "nameAr": "تعفن الثمار الناضجة وتحنطها",
               "CF": 0.9967463053154344
            },
            {
               "name": "fruits have purple/orange color",
               "nameAr": "ظهور اللون الوردي/البرتقالي على سطح الثمار المتعفنة",
               "CF": 0.4754554953695336
            },
            {
               "name": "Leaves dry",
               "nameAr": "ذبول و جفاف الاوراق و موتها",
               "CF": 0.4755455808113952
            },
            {
               "name": "Dry branches",
               "nameAr": "ذبول و جفاف الأفرع و موتها",
               "CF": 0.48093947709108553
            }
         ]
      },
      {
         "name": "نقص المغنيزيوم Magnesium  Deficiency\n",
         "link": "https://www.al-hakem.com/نقص-المغنيزيوم-على-الزيتون/",
         "details": "",
         "symptoms": [
            {
               "name": "Leaf edges chlorosis",
               "nameAr": "إصفرار حواف الأوراق، يمتد إلى أنسجة الورقة بين العروق",
               "CF": 0.9832399778533228
            },
            {
               "name": "Leaf edges turns brown",
               "nameAr": "تتحول حواف الأوراق إلى اللون البني",
               "CF": 0.24502511214437284
            },
            {
               "name": "Leaves Die",
               "nameAr": "موت الاوراق",
               "CF": 0.9040036079226494
            },
            {
               "name": "leaves fall",
               "nameAr": "تساقط الأوراق",
               "CF": 0.706189315866991
            }
         ]
      }
   ],
   "تفاح - Apple": [
      {
         "name": "دودة ثمار التفاح أو الكمثرى Codling Moth\n",
         "link": "https://www.al-hakem.com/دودة-ثمار-التفاح-أو-الكمثرى/",
         "details": "الإسم العلمي : CYDIA( CARPOCAPSA) POMONELLA",
         "symptoms": [
            {
               "name": "presence of digging larvae in the skin of the fruits",
               "nameAr": "وجود حفر لليرقات في جلد الثمار",
               "CF": 0.6931088798168374
            },
            {
               "name": "The presence of the larva inside the fruit to the heart region (pulp) and the infection of the seeds in the fruit",
               "nameAr": "وجود اليرقة داخل الثمرة إلى منطقة القلب(اللب) وإصابة البذور الموجودة في الثمرة",
               "CF": 0.6381772398500639
            },
            {
               "name": "Holes in the fruit whose openings are covered by larval stool",
               "nameAr": "يمكن مشاهدة ثقوب في الثمرة يغطي فتحاتها البراز اليرقي",
               "CF": 0.6334486615938423
            },
            {
               "name": "It is observed that the larvae enter from the sides of the fruit, from its contact area with the fruit holder, from the cup or from the end of the fruit.",
               "nameAr": "يلاحظ دخول اليرقات من جوانب الثمرة أو من منطقة اتصالها بحامل الثمرة أو من الكأس أو من نهاية الثمرة.",
               "CF": 0.9371125672830536
            }
         ]
      },
      {
         "name": "جفاف وموت الاطراف DIE- BACK \n",
         "link": "https://www.al-hakem.com/جفاف-وموت-الاطراف-في-التفاح/",
         "details": "العامل المسبب : FUSARIUM ACUMINATUM",
         "symptoms": [
            {
               "name": "Peripheral branch wounds",
               "nameAr": "جروح الأفرع الطرفية",
               "CF": 0.22407251234965542
            },
            {
               "name": "dark brown color branches",
               "nameAr": "تحول لون الأفرع إلى اللون البني الداكن",
               "CF": 0.4196014609626967
            },
            {
               "name": "dry branches",
               "nameAr": "جفاف الأفرع وخاصة الطرفية",
               "CF": 0.17544415042954475
            },
            {
               "name": "disease fall down to branches root",
               "nameAr": "تمتد الإصابة إلى أسفل في اتجاه قاعدة الأفرع",
               "CF": 0.3377422870336647
            },
            {
               "name": "Create a crimson-colored ring between the affected and healthy area",
               "nameAr": "تكوين حلقة قرمزية اللون بين المنطقة المصابة والسليمة.",
               "CF": 0.3170086665285715
            }
         ]
      },
      {
         "name": "نقص البورون Boron",
         "link": "https://www.al-hakem.com/نقص-البورون-على-التفاح/",
         "details": "ممكن تصحيح نقص البورون بواسطة تطبيق للتربة أو رش رذاذ ورقي. ولكن لا ينبغي استخدام علاجات الرش في نفس الموسم كتطبيق للتربة بسبب خطر السمية.",
         "symptoms": [
            {
               "name": "in case of flowering succeed, then the flowers will wither and die",
               "nameAr": "بحال نجاح الأزهار فيحدث ذبول وموت للأزهار",
               "CF": 0.3100314398689018
            },
            {
               "name": "normal growth leaves",
               "nameAr": "تظل الأوراق نامية بطريقة طبيعية،",
               "CF": 0.010202840782180367
            },
            {
               "name": "Corks areas in fruit.",
               "nameAr": "تكون مناطق فلينية في الثمرة.",
               "CF": 0.7067363185145409
            },
            {
               "name": "leaves die",
               "nameAr": "موت الأوراق",
               "CF": 0.15
            },
            {
               "name": "blossom Failed to grow",
               "nameAr": "فشل النبات من تكوين براعم جديدة",
               "CF": 0.15
            },
            {
               "name": "Tree death",
               "nameAr": "موت الاشجار",
               "CF": 0.6361184433984247
            }
         ]
      },
      {
         "name": "مرض التدرن التاجي  Crown gall \n",
         "link": "https://www.al-hakem.com/التدرن-التاجي-على-التفاح/",
         "details": "المسبب :بكتيريا Agrobacterium tumefaciens (Smith and Townsend)Conn\nسمي هذا المرض بالتدرن التاجى لأن الأورام أو التدرنات التى تسببها البكتيريا كثيرا ما تتكون فى منطقة التاج لشتلات وأشجار التفاحيات المصابة ومن الممكن أيضا حدوث هذه الأورام في أي مكان بالمجموع الجذري عندما تكون الإصابة شديدة ولكن يجب ألا نخلط بين هذه الأورام والعقد النيماتودية المتسببة عن نيماتودا تعقد الجذور\nالبكتيريا المسببة لهذا المرض تعتبر من الميكروبات التي تعيش في التربة وتنتشر بدرجة كبيرة في مشاتل الفاكهة\nتسبب مشاكل وخسائر اقتصادية حيث أن جميع الشتلات المصابة بهذا المرض يجب التخلص منها بالأعدام حرقا وعدم تداولها لاستعمالها في زراعة وإنشاء بساتين",
         "symptoms": [
            {
               "name": "Bacteria enter the host's tissues in the roots",
               "nameAr": "تدخل البكتيريا أنسجة العائل عن طريق الجروح ولا تستطيع إصابة المجموع الجذري الخالي من الجروح",
               "CF": 0.7312704689502626
            },
            {
               "name": "root shows small, light-colored spheres with soft feel",
               "nameAr": "يظهر على المجموع الجذري المصاب تدرنات صغيرة فاتحة اللون تأخذ شكل كروي وملمس طري",
               "CF": 0.7897536256242266
            },
            {
               "name": "root shows big spheres and its surface takes an irregular shape",
               "nameAr": "يظهر على المجموع الجذري تدرنات كبيرة في الحجم ويأخذ سطحها شكل غير منتظم",
               "CF": 0.8657844544770339
            },
            {
               "name": "The tumor's outer tissues are dark brown",
               "nameAr": "تحول لون الأنسجة الخارجية للورم إلى اللون البنى الداكن",
               "CF": 0.07854548535979544
            },
            {
               "name": "Root tumors decompositiont",
               "nameAr": "ظهور تحلل على أورام الجذور",
               "CF": 0.7063347880532331
            }
         ]
      },
      {
         "name": "الأشنة في التفاحيات",
         "link": "https://www.al-hakem.com/الأشنة-على-التفاح/",
         "details": "عبارة عن نموات حرشفية مختلفة الألوان على الأفرع والنموات الحديثة تزداد بزيادة الرطوبة والحرارة . و تتكون من نمو فطري وطحلبي يعيشان معا تكافليا .",
         "symptoms": [
            {
               "name": "Thin yellow or green plates appear on branches and stems",
               "nameAr": "ظهور صفائح رفيعة ذات لون أصفر أو أخضر على الأوراق",
               "CF": 0.6895292453267832
            },
            {
               "name": "decreased in the crop",
               "nameAr": "قلة ونقص في كمية المحصول الناتج",
               "CF": 0.13380516745199522
            },
            {
               "name": "weakness in plant growth",
               "nameAr": "ضعف عام في نمو الشجرة والنبات",
               "CF": 0.8029447516623651
            },
            {
               "name": "farming in narrow places ",
               "nameAr": "الزراعة على مسافات ضيقة",
               "CF": 0.0011291123662768854
            }
         ]
      },
      {
         "name": "نقص النتروجين Nitrogen deficiency\n",
         "link": "https://www.al-hakem.com/نقص-النتروجين-على-التفاح/",
         "details": "",
         "symptoms": [
            {
               "name": "smaller leaves",
               "nameAr": "صغر حجم الأوراق ",
               "CF": 0.41472040274911703
            },
            {
               "name": "slow growth branches",
               "nameAr": "بطء نمو الأفرع والأغصان",
               "CF": 0.3789774145536188
            },
            {
               "name": "Few vegetation sum",
               "nameAr": "قلة المجموع الخضري بصفة عامة.",
               "CF": 0.5587329904426873
            },
            {
               "name": "Lack of knots of the fruits",
               "nameAr": "قلة عقد الثمار",
               "CF": 0.2
            },
            {
               "name": "small fruits",
               "nameAr": "صغر حجم الثمار.",
               "CF": 0.2
            }
         ]
      },
      {
         "name": "العفن الأرميلاري Armillaria Root Rot\n",
         "link": "https://www.al-hakem.com/العفن-الأرميلاري-على-التفاح/",
         "details": "العامل المسبب : Armillaria mellea",
         "symptoms": [
            {
               "name": "weakness in plant growth",
               "nameAr": "ضعف عام في النبات والنمو",
               "CF": 0.20273929242052602
            },
            {
               "name": "yellowish leaves",
               "nameAr": "إصفرار الأوراق",
               "CF": 0.42454649695986524
            },
            {
               "name": "leaves fall",
               "nameAr": "تساقط الأوراق",
               "CF": 0.0951043470778079
            },
            {
               "name": "die-back",
               "nameAr": "موت الأفرع الطرفية Die-Back",
               "CF": 0.15
            },
            {
               "name": "Small white spots on roots",
               "nameAr": "عند نزع قلف الأشجار المصابة في منطقة الجذوع القريبة من سطح التربة يلاحظ وجود نموات ميسليوية مروحية الشكل بيضاء اللون",
               "CF": 0.2280936142946567
            }
         ]
      },
      {
         "name": "منّ التفاح الوردي والأخضر منّ التفاح الوردي والأخضر\n",
         "link": "https://www.al-hakem.com/منّ-التفاح-الوردي-والأخضر/",
         "details": "الإسم العلمي  DYSAPHIS PLANTAGINEA\nيعتبر أكثر الأنواع ضررا على أشجار التفاح وقد تختلف الإصابة من سنة إلى أخرى، وعند اشتداد الإصابة يحدث نقصا خطيرا بالمحصول.",
         "symptoms": [
            {
               "name": "stop growth fruits",
               "nameAr": "توقف نمو الثمار",
               "CF": 0.9942660090486267
            },
            {
               "name": "leaves curl and wrap around",
               "nameAr": "تجعد الأوراق المصابة والتفافها حول نفسها",
               "CF": 0.409199244830989
            },
            {
               "name": "Pink aphids were present in groups on fuites leaves",
               "nameAr": "تواجد المن الوردي في تجمعات فوق أوراق حامل الثمرة.",
               "CF": 0.4812643594260455
            },
            {
               "name": "Honey Symposium",
               "nameAr": "تكون الندوة العسلية ونمو فطر العفن الاسود.",
               "CF": 0.6161904518389819
            },
            {
               "name": "Distorted fruits",
               "nameAr": "تشوه الثمار وعدم صلاحيتها للتسويق",
               "CF": 0.1713179567652342
            }
         ]
      },
      {
         "name": "من التفاح القطني Apple woolly aphid\n",
         "link": "https://www.al-hakem.com/من-التفاح-القطني-2/",
         "details": "تعتبر حشرة مَنْ التفاح القطني آفة خطيرة في كثير من المزارع والبلدان المنتجة لأصناف التفاح مما يتسبب في انخفاض نمو الأشجار وتلوث الفاكهة والاضرار بالانتاج . وهي تغزو جميع أجزاء الشجرة حتى الجذور .\nهذه الحشرة لونها إرجواني باهت بالنسبة للذكور واما الاناث فيكون لونها أخضر باهت .",
         "symptoms": [
            {
               "name": "Tumors on leg",
               "nameAr": "حدوث تورمات في الساق وعلى الجذور نتيجة امتصاص المن لعصارة النبات التي تؤدي إلى تهيج الأنسجة , وعند الضغط عليها يظهر سائل أحمر اللون .",
               "CF": 0.5
            },
            {
               "name": "Honey Symposium",
               "nameAr": "تكون الندوة العسلية ونمو فطر العفن الاسود.",
               "CF": 0.08215996464877051
            }
         ]
      },
      {
         "name": "نقص المغنيزيوم Magnesium  Deficiency\n",
         "link": "https://www.al-hakem.com/نقص-المغنيزيوم-على-التفاح/",
         "details": "",
         "symptoms": [
            {
               "name": "Leaf edges chlorosis",
               "nameAr": "إصفرار حواف الأوراق، يمتد إلى أنسجة الورقة بين العروق",
               "CF": 0.03418004252583684
            },
            {
               "name": "leaves die",
               "nameAr": "موت الأوراق",
               "CF": 0.5245426256976132
            },
            {
               "name": "Leaf edges turns brown",
               "nameAr": "تتحول حواف الأوراق إلى اللون البني",
               "CF": 0.5247360980760127
            },
            {
               "name": "leaves fall",
               "nameAr": "تساقط الأوراق",
               "CF": 0.6882688525553408
            }
         ]
      },
      {
         "name": "خنافس القلف BARK BEETLE\n",
         "link": "https://www.al-hakem.com/خنافس-القلف-على-التفاح/",
         "details": "الإسم العلمي:\nECCOPTOGAATER (SCOLYTUS) AEGEYPTIACUS\nخنافس صغيرة الحجم (2- 2,5 مم) ولونها بني قاتم أو أسود ومن أهم عوائلها الخوخ والمشمش والبرقوق واللوز والتفاح والكمثرى والرمان والموالح.",
         "symptoms": [
            {
               "name": "halls on branches",
               "nameAr": "وجود ثقوب صغيرة على جذوع أو أفرع الأشجار المصابة",
               "CF": 0.5933067917417532
            },
            {
               "name": "presence of digging larvae in the skin of the fruits",
               "nameAr": "وجود حفر لليرقات في جلد الثمار",
               "CF": 0.4091661491467107
            },
            {
               "name": "blossom and new small Leaves dry",
               "nameAr": "جفاف البراعم والأوراق الصغيرة الحديثة",
               "CF": 0.27750393484032454
            },
            {
               "name": "dry branches",
               "nameAr": "جفاف الأفرع وخاصة الطرفية",
               "CF": 0.04117631623205398
            },
            {
               "name": "Tree death",
               "nameAr": "موت الاشجار",
               "CF": 0.7577767637336879
            }
         ]
      },
      {
         "name": "تقرح الجذور",
         "link": "https://www.al-hakem.com/تقرح-الجذور-في-التفاح/",
         "details": "العامل المسبب : نيماتودا التقرح (Lesion Nematodes (Pratylenchus spp\nتعتبر نيماتودا التقرح Pratylenchus spp من اهم اجناس النيماتودا التي تتطفل وتتجول داخل الجذور Endo-Migratory Nematodes. وهي تنتشر في جميع أنحاء العالم تقريبا\nتعتبر بما تسببه من تقر حات شديدة للجذور مسؤولة عن تدهور المجموع الجذري، وبالتالي حساسية النباتات المصابة للجفاف، مما يؤدي إلى انخفاض كبير في النمو الخضري والإنتاج.",
         "symptoms": [
            {
               "name": "weakness in plant growth",
               "nameAr": "ضعف عام في النبات والنمو",
               "CF": 0.8435242997493964
            },
            {
               "name": "smaller leaves",
               "nameAr": "صغر حجم الأوراق",
               "CF": 0.6879206979587615
            },
            {
               "name": "Paleness leaves",
               "nameAr": "شحوب في لون الأوراق",
               "CF": 0.8509469568685202
            },
            {
               "name": "die-back",
               "nameAr": "موت الأفرع الطرفية Die-Back",
               "CF": 0.9339564921635045
            },
            {
               "name": "root with rectangular botch spots",
               "nameAr": "يوجد على الجذر بقع متقرحة مستطيلة نوعاً ما بنية اللون تزداد في الحجم مع اشتداد الإصابة وتتحول للون الأسود",
               "CF": 0.84
            },
            {
               "name": "Root decomposition",
               "nameAr": "تحلل الجذر",
               "CF": 0.5
            }
         ]
      },
      {
         "name": "حفار ساق التفاح The Leopard Moth\n\n",
         "link": "https://www.al-hakem.com/حفار-ساق-التفاح/",
         "details": "العامل المسبب : حشرة Zeuzera pyrina\nتخرج الحشرات البالغة من العذارى ابتداء من شهر أيار وحتى حزيران .تضع الأنثى البيض داخل الشقوق والجروح والثقوب الموجودة على سيقان أفرع الأشجار, يفقس البيض بعد حوالي 10أيام إلى يرقات تبدأ في حفر أنفاقها مباشرة داخل الأفرع . تستمر اليرقات فى التغذية والحفر داخل السيقان والأفرع لمدة تزيد عن عشرة أشهر. تتحول بعدها إلى عذراء داخل النفق , ويستمر طور العذراء حوالى 20يوم . تخرج بعدها الحشرة الكاملة من خلال ثقب في نهاية النفق على الساق أو الفروع ,تاركة جلد العذراء بارزا من الثقب . حيث يتم التلقيح ووضع البيض بعد الخروج مباشرة .وتبلغ فترة حياة الفراشة 5أيام ولها جيل واحد في العام.",
         "symptoms": [
            {
               "name": "skin on branches ",
               "nameAr": "يوجد جلود لعذارى الحشرة بارزة على سيقان وأفرع الأشجار",
               "CF": 0.7576932803512546
            },
            {
               "name": "halls on branches",
               "nameAr": "وجود ثقوب كثيرة على أفرع الشجرة",
               "CF": 0.9037369125057807
            },
            {
               "name": "Sawdust conversion holes",
               "nameAr": "ثقوب تحوي نشارة خشب مخلوطا ببراز اليرقات الموجودة داخل النفق",
               "CF": 0.5635510406195869
            },
            {
               "name": "break branches",
               "nameAr": "تتكسر الأفرع المصابة",
               "CF": 0.4480955497694684
            },
            {
               "name": "dry branches",
               "nameAr": "ذبول وجفاف الأفرع",
               "CF": 0.9054274606536298
            },
            {
               "name": "Tree death",
               "nameAr": "موت الاشجار",
               "CF": 0.2
            },
            {
               "name": "decreased in the crop",
               "nameAr": "قلة ونقص في كمية المحصول الناتج",
               "CF": 0.1
            }
         ]
      },
      {
         "name": "لسعة الشمس  SUN SCALD\n",
         "link": "https://www.al-hakem.com/لسعة-الشمس-على-التفاح/",
         "details": "نتيجة لتعرض الثمار لأشعة الشمس المباشرة",
         "symptoms": [
            {
               "name": "areas exposed to the sun of fruit turn to a dark color",
               "nameAr": "تتحول المناطق المعرضة للشمس من الثمرة إلى اللون الغامق",
               "CF": 0.5277754266450729
            },
            {
               "name": "Fruit is easy to cut from the tree",
               "nameAr": "يسهل نزع الثمار من الشجرة",
               "CF": 0.5564770637749241
            },
            {
               "name": "After collection, fruits turn dark brown",
               "nameAr": "بعد الجمع تتحول الثمار المصابة إلى اللون البني الغامق",
               "CF": 0.5786109612006386
            },
            {
               "name": "Surface cracks on the fruit",
               "nameAr": "يلاحظ وجود شقوق سطحية على الثمرة",
               "CF": 0.6456984230124126
            }
         ]
      },
      {
         "name": "نقص الحديد Iron Deficiency\n",
         "link": "https://www.al-hakem.com/نقص-الحديد-في-التفاح/",
         "details": "",
         "symptoms": [
            {
               "name": "soil alkaline",
               "nameAr": "التربة قلوية",
               "CF": 0.5298596119382504
            },
            {
               "name": "yellowish leaves",
               "nameAr": "إصفرار الأوراق",
               "CF": 0.4
            },
            {
               "name": "yellowish leaf edges",
               "nameAr": "تحول الأوراق الطرفية إلى اللون الأصفر",
               "CF": 0.6
            },
            {
               "name": "white leaf edges",
               "nameAr": "تحول الأوراق الطرفية إلى اللون الأبيض",
               "CF": 0.9
            }
         ]
      },
      {
         "name": "مرض وجه القطة Cat-facing Insects\n",
         "link": "https://www.al-hakem.com/مرض-وجه-القطة-على-التفاح/",
         "details": "العامل المسبب : Lygus Bug , stink bug, and boxelder bug",
         "symptoms": [
            {
               "name": "Insect spread",
               "nameAr": "انتشار الحشرات",
               "CF": 0.7314917025216677
            },
            {
               "name": "Ugly, deformed, and scarred fruit",
               "nameAr": "فاكهه قبيحة مشوهه و مندبة",
               "CF": 0.4604062742677838
            },
            {
               "name": "The appearance of aphids on the fruits",
               "nameAr": "ظهور المن على الثمار",
               "CF": 0.8631492141041184
            },
            {
               "name": "Distorted fruits",
               "nameAr": "تشوه الثمار وعدم صلاحيتها للتسويق",
               "CF": 0.8016123153912436
            }
         ]
      },
      {
         "name": "العفن التاجي Phytophthora crown and root rot \n",
         "link": "https://www.al-hakem.com/العفن-التاجي-على-التفاح/",
         "details": "العامل المسبب : Phytophthora spp",
         "symptoms": [
            {
               "name": "yellowish leaves",
               "nameAr": "إصفرار الأوراق",
               "CF": 0.7419648531306176
            },
            {
               "name": "leaves fall",
               "nameAr": "تساقط الأوراق",
               "CF": 0.9453802605520569
            },
            {
               "name": "root colored red to dark brown",
               "nameAr": "يتلون قلف الجذور الرئيسية المصابة باللون المائل للأحمرار إلى اللون البني الغامق",
               "CF": 0.9139061641464935
            },
            {
               "name": "Cracks in the crown",
               "nameAr": "تشققات في منطقة التاج",
               "CF": 0.5098940774543052
            },
            {
               "name": "Tree death",
               "nameAr": "موت الاشجار",
               "CF": 0.44147509950905484
            }
         ]
      },
      {
         "name": "نقص الفوسفور Phosphorus deficiency\n",
         "link": "https://www.al-hakem.com/نقص-الفوسفور-على-التفاح/",
         "details": "",
         "symptoms": [
            {
               "name": "Paleness of the vegetative sum color.",
               "nameAr": "شحوب لون المجموع الخضري.",
               "CF": 0.4011406845110158
            },
            {
               "name": "Leaf edges soft",
               "nameAr": "تموج حواف الأوراق وتصبح رخوة وذابلة وتصاب بالبرقشة",
               "CF": 0.018945895505969035
            },
            {
               "name": "small branches",
               "nameAr": "أفرع صغيرة",
               "CF": 0.15
            },
            {
               "name": "smaller leaves",
               "nameAr": "صغر حجم الأوراق",
               "CF": 0.2
            },
            {
               "name": "die-back",
               "nameAr": "موت الأفرع الطرفية Die-Back",
               "CF": 0.1
            },
            {
               "name": "decreased in the crop",
               "nameAr": "قلة ونقص في كمية المحصول الناتج",
               "CF": 0.29459624807828577
            },
            {
               "name": "black branches",
               "nameAr": "تموت الأفرع الصغيرة وتتحول إلى اللون الأسود.",
               "CF": 0.3
            }
         ]
      },
      {
         "name": "مرض التقرح البكتيري Pear Canker\n\n",
         "link": "https://www.al-hakem.com/التقرح-البكتيري-على-التفاح/",
         "details": "تنتج عن خلل فسيولوجى في الشجرة إلا أنه تم عزل بعض المسببات المرضية الفطرية التي تسبب ظهور هذه التقرحات من الأفرع الطرفية مثل Diplodia Metalensis .",
         "symptoms": [
            {
               "name": "Circular ulcers on branches",
               "nameAr": "تقرحات دائرية الشكل على الأفرع والأذرع الرئيسية للشجرة",
               "CF": 0.6254886265181788
            },
            {
               "name": "dry branches",
               "nameAr": "جفاف الأفرع وخاصة الطرفية",
               "CF": 0.3644129989361927
            },
            {
               "name": "die-back",
               "nameAr": "موت الأفرع الطرفية Die-Back",
               "CF": 0.3
            }
         ]
      }
   ],
   "طماطم - tomatoes": [
      {
         "name": "حلم صدأ الطماطم الدودي Tomato rust mite",
         "link": "https://www.al-hakem.com/حلم-صدأ-الطماطم-الدودي/",
         "details": "الإسم العلمي( Aculops lycopersici (Massee))\nتتحد البقع على الأوراق وتجف وتموت",
         "symptoms": [
            {
               "name": "Withering and drying of leaves occurs from the bottom to the top of the plant.",
               "nameAr": "يحدث ذبول الاوراق وجفافها من اسفل إلى أعلى النبات.",
               "CF": 0.8277279701807106
            },
            {
               "name": "The lower leaves are painted in silver, which then turns bronze",
               "nameAr": "تلون الأوراق السفلية باللون الفضي الذي يتحول بعد ذلك إلى اللون البرونزي",
               "CF": 0.23611296066980791
            },
            {
               "name": "Withering leaves",
               "nameAr": "ذبول الأوراق",
               "CF": 0.9343371602997829
            },
            {
               "name": "leaves fall",
               "nameAr": "سقوط الأوراق",
               "CF": 0.37095481865895796
            },
            {
               "name": "Fruits subjected to sunburn",
               "nameAr": "تعرض الثمار إلى حروق الشمس",
               "CF": 0.5602395219134375
            },
            {
               "name": " fruits look rough and rusty in color, with some cracks on the neck",
               "nameAr": "تبدو الثمار خشنة الملمس وصدئية اللون مع وجود بعض التشققات عند منطقة العنق",
               "CF": 0.9845007484104418
            }
         ]
      },
      {
         "name": "نيماتودا تعقد الجذور Root Knot",
         "link": "https://www.al-hakem.com/نيماتودا-تعقد-الجذور-على-الطماطم/",
         "details": "العامل المسبب : النيماتودا Meloidogyne spp",
         "symptoms": [
            {
               "name": "Abundant root and odor multiply over the point of infection",
               "nameAr": "تكاثر جذور غزير ورائحة فوق نقطة العدوى",
               "CF": 0.06687009248295539
            },
            {
               "name": "On the aerial parts of the plant, it appears stunting, targling and wilting",
               "nameAr": "تشمل الأعراض فوق الأرض على الأجزاء الهوائية التقزم والتبرقش والذبول.",
               "CF": 0.6821201692827112
            },
            {
               "name": "the crop is small",
               "nameAr": "المحصول الناتج يكون صغيرا بشكل عام وقد يكون غير قابل للتسويق.",
               "CF": 0.18089695065230238
            },
            {
               "name": "plant death",
               "nameAr": "الموت المبكر للنبات",
               "CF": 0.8372987030519292
            },
            {
               "name": "Tuberculosis is infected and shows deformities",
               "nameAr": "إصابة الدرنات بالعدوى وتظهر بها غورات أو تشوهات",
               "CF": 0.09733649975201664
            }
         ]
      },
      {
         "name": "مرض العفن الرمادي Gray model disease ",
         "link": "https://www.al-hakem.com/مرض-العفن-الرمادي-في-الطماطم/",
         "details": "العامل المسبب : الفطر Botrytis cinerea\nينتشر المرض في الشتاء حيث الرطوبة العالية وخاصة داخل البيوت البلاستيكية الغير مغطاة، وكذلك أثناء عمليات النقل والتخزين",
         "symptoms": [
            {
               "name": "Yellow spots appear on the top surface of leaves, spores, and fungus carriers that give the appearance of gray mold",
               "nameAr": "ظهور بقع صفراء اللون على السطح العلوي للأوراق وجراثيم وحوامل الفطر التي تعطي مظهر العفن الرمادي",
               "CF": 0.5762163825396946
            },
            {
               "name": "Clear sores appear on the leg with overlapping rings.",
               "nameAr": "تظهر تقرحات واضحة على الساق ذات حلقات متداخلة.",
               "CF": 0.676708750637184
            },
            {
               "name": "Mold and stains appear on ripe fruits in the form of light brown or gray stains saturated with water",
               "nameAr": "عفن وتبقع على الثمار الناضجة على شكل بقع بنية فاتحة أو رمادية ومتشبعة بالماء",
               "CF": 0.1736060360590692
            },
            {
               "name": "fall fruit",
               "nameAr": "غالبا ما تسقط الثمار المصابة بشدة",
               "CF": 0.42576645995015305
            },
            {
               "name": "fruits are rots",
               "nameAr": "تتعفن الثمرة",
               "CF": 0.05687960322764318
            }
         ]
      },
      {
         "name": "سقوط البادرات Dampinf -off",
         "link": "https://www.al-hakem.com/سقوط-البادرات-في-الطماطم/",
         "details": "العامل المسبب: فطريات التربة ومن أهمها:\nRhizoctonia solni .spp Pythium\nPhytophthora SPP. Thielaviopsis basicola",
         "symptoms": [
            {
               "name": "Seed infection occurs before germination",
               "nameAr": "تحدث الإصابة المرضية للبذور قبل إنباتھا",
               "CF": 0.4941211362361345
            },
            {
               "name": "Soil fungi penetrate the outer shell of the seed",
               "nameAr": "تخترق الفطريات الموجودة بالتربة الغلاف الخارجي للبذرة",
               "CF": 0.07631031329559068
            },
            {
               "name": "Fetal rot, which leads to non-germination",
               "nameAr": "تعفن الجنين مما يؤدى الى عدم الإنبات",
               "CF": 0.18550167193994016
            }
         ]
      },
      {
         "name": "مرض وجة القط cat face",
         "link": "https://www.al-hakem.com/مرض-وجه-القط-في-الطماطم/",
         "details": "مرض وجه القط من الامراض الفسيولوجية التى تصيب الطماطم",
         "symptoms": [
            {
               "name": "The affected fruits appear to have large bends and protrusions",
               "nameAr": "تبدو الثمار المصابة بها إنحناءات و بروزات كبيرة",
               "CF": 0.016787822949925535
            },
            {
               "name": "The fruits are crowded at the pink end",
               "nameAr": "تتزاحم الثمار عند الطرف الزهري",
               "CF": 0.7434614153879204
            },
            {
               "name": "Blossoms and fruit hold in cold weather",
               "nameAr": "إزهار و عقد الثمار في الجو البارد.",
               "CF": 0.512255241564687
            },
            {
               "name": "deep gaps Between the fruits extend into the Plant bean",
               "nameAr": "تمتد بين الثمار فجوات عميقة الى داخل الثمرة",
               "CF": 0.9382201358056692
            }
         ]
      },
      {
         "name": "لفحة الشمس Sun Scald",
         "link": "https://www.al-hakem.com/لفحة-الشمس-على-الطماطم/",
         "details": "",
         "symptoms": [
            {
               "name": "White spots appear on the green fruits and then turn pale yellow when ripe",
               "nameAr": "ظهور بقع بيضاء على الثمار الخضراء ثم تتحول للون الاصفر الباهت عند النضج.",
               "CF": 0.8
            }
         ]
      },
      {
         "name": "دودة اللوز الاميركية armigeera heliothis",
         "link": "https://www.al-hakem.com/دورة-ثمار-الطماطم/",
         "details": "الإسم العلمي :armigera Heliothis\nيتراوح طول دودة ثمار وأزهار البندورة بين 15 و40 ملم. قد يكون لون الدودة بنيا مسودا ، أخضرا أو أصفر، وتتميزبوجود خط غامق في وسط ظهرها وعلى جانبيها خطان افتح لونا يليهما خطان أغمق منهما، ورأس الدودة أصفر مائل إلى البني أما طول الفراشة فهو بين 30 و40 ملم.",
         "symptoms": [
            {
               "name": "Reproductive organs are damaged in plants",
               "nameAr": "تلف اعضاء التكاثر في النباتات مثل البراعم والازهار",
               "CF": 0.44018783917242
            },
            {
               "name": "fruits are distinguished by the presence of a round nozzle",
               "nameAr": "تتميز الاصابة على الثمار بوجود فوهة الثقوب دائرية",
               "CF": 0.693872940611147
            },
            {
               "name": "Fruit spoilage and early discoloration, causing crop losses",
               "nameAr": "تلف الثمار وتلونها مبكراً مسببة خسائر في الانتاج",
               "CF": 0.667481021917095
            },
            {
               "name": "fruits are rots",
               "nameAr": "تتعفن الثمرة",
               "CF": 0.7917091668256522
            },
            {
               "name": "Large holes on the surface of the affected fruit are accompanied by wear of the tissue of the fruit",
               "nameAr": "ثقوب كبيرة على سطح الثمرة المصابة مصحوبة بإهتراء أنسجة الثمرة",
               "CF": 0.6450189227161486
            }
         ]
      },
      {
         "name": "الدودة القارضة Cut worm",
         "link": "https://www.al-hakem.com/الدودة-القارضة-على-الطماطم/",
         "details": "الإسم العلمي :  Agrotishg ipsilon\nتعتبر الديدان القارضة من الآفات الهامة التي تسبب خسائر كبيرة للبادرات حيث يضطر المزارع عند اشتداد الإصابة الى إعادة الزراعة مرة أخرى مما يؤدى الى تأخر نمو النبات وعدم حصوله على احتياجاته الحرارية كاملة مما يؤدى الى قلة المحصول .\nيعود سبب تسمية يرقات هذه الحشرة بالدودة القارضة الى أن هذه اليرقات فى أطوار نموها الأخيرة تكون غير قادرة على تسلق النباتات نظرا لخلو أرجلها البطنية الكاذبة من الخطاطيف وعلى ذلك فان هذه اليرقات تكتفي بالزحف على سطح التربة فقط وتتغذى على ما يقابلها من سيقان النباتات الغضة وغالبا ما يكون القرض كامل .",
         "symptoms": [
            {
               "name": "The infection appears suddenly and in splattered spots in the field",
               "nameAr": "ظهور الإصابة فجأة وفي بقع متناثرة في الحقل",
               "CF": 0.9592890314255135
            },
            {
               "name": "The presence of seedlings legs tilted above the soil surface",
               "nameAr": "وجود سيقان البادرات مائلة فوق سطح التربة",
               "CF": 0.013044517098935282
            },
            {
               "name": "There are some paper crumbs near the seedlings",
               "nameAr": "وجود بعض فتات الأوراق بجوار البادرات المقروضة",
               "CF": 0.34394813865497276
            }
         ]
      },
      {
         "name": "عفن الرقبة أو اللفحة المبكرة",
         "link": "https://www.al-hakem.com/عفن-الرقبة-أو-اللفحة-المبكرة-على-الطما/",
         "details": "العامل المسبب: الفطر Alternaria solani\n\nيحدث و يتطور في الجو الرطب و يؤدي المرض الى عفن الرقبة سواء في المشتل أو الدفيئة فان الخسائر تكون كبيرة وقد يسبب تساقط كبير للأوراق مما يؤدي الى صغر حجم الثمار وقلة عددها و هو عادة يصيب النباتات الضعيفة .",
         "symptoms": [
            {
               "name": "A small black scar to brown, quickly grows up and surrounds the leg",
               "nameAr": "ندبة صغيرة سوداء الى بنية سرعان ما تكبر وتحيط بالساق",
               "CF": 0.5824730816039269
            },
            {
               "name": "spots on leaves",
               "nameAr": " بقع على الأوراق",
               "CF": 0.4289495561618588
            },
            {
               "name": "Leaves dry",
               "nameAr": "جفاف الاوراق",
               "CF": 0.2
            },
            {
               "name": "leaves die",
               "nameAr": "موت الاوراق",
               "CF": 0.2
            },
            {
               "name": "The plant becomes bare and the fruits are exposed to sunstroke",
               "nameAr": "يصبح النبات اجرد و تتعرض الثمار لضربة الشمس",
               "CF": 0.34305851636842666
            },
            {
               "name": "Plant death, and if it lives, its growth and production will deteriorate",
               "nameAr": "موت النبات وإذا عاش فان نموه وإنتاجه يتدهور",
               "CF": 0.5359031519774581
            },
            {
               "name": "fall fruit",
               "nameAr": "غالبا ما تسقط الثمار المصابة بشدة",
               "CF": 0.7779169438241315
            }
         ]
      },
      {
         "name": "اللفحة المتأخرة على الطماطم Late Blight",
         "link": "https://www.al-hakem.com/اللفحة-المتأخرة-على-الطماطم/",
         "details": "العامل المسبب :الفطر phytophthora infestans\nينتشر عادة في الجو البارد الماطر ويسبب خسائر فادحة وقد يقضي على الحقل كله خلال فترة وجيزة إذا لم يعالج .\n",
         "symptoms": [
            {
               "name": "Leaves bent down",
               "nameAr": "إنحناء الأوراق للأسفل",
               "CF": 0.17458660029319395
            },
            {
               "name": "Dark green watery spots on the top surface of the leaves turn brown or black in cold, humid weather",
               "nameAr": "ظهور بقع مائية مخضرة داكنة لها لمعة الزيت على السطح العلوي للأوراق تتحول هذه البقع للون البني أو المسود في الجوالبارد الرطب",
               "CF": 0.2687221965897807
            },
            {
               "name": "The paper turns brown or blackish",
               "nameAr": "تتحول الورقة كلها الى اللون البني أو المسود",
               "CF": 0.5647732988949238
            },
            {
               "name": "Leaves dry",
               "nameAr": "جفاف الاوراق",
               "CF": 0.2
            },
            {
               "name": "leaves die",
               "nameAr": "موت الاوراق",
               "CF": 0.2
            },
            {
               "name": "On the leg and leaf necks there are dark brown or dark spots",
               "nameAr": "تكون على الساق وأعناق الأوراق بقع بنية داكنة أو مسودة",
               "CF": 0.14365387023410348
            },
            {
               "name": "Leg dryness and death",
               "nameAr": "جفاف وموت الساق",
               "CF": 0.31911824529614097
            },
            {
               "name": "Fruits appear on olive black spots that may take the color of bitumen, usually dry, hardened, shallow, and irregular",
               "nameAr": "يظهر على الثمار بقع سوداء زيتونية وقد تأخذ لون القار وعادة ما تكون جافة ومتصلبة وسطحية وغير منتظمة",
               "CF": 0.3766017774549151
            }
         ]
      },
      {
         "name": "اللفحة الجنوبية\nSouthern Blight\n",
         "link": "https://www.al-hakem.com/اللفحة-الجنوبية-على-الطماطم/",
         "details": "",
         "symptoms": [
            {
               "name": "plant death",
               "nameAr": "موت النبات",
               "CF": 0.6003978058771997
            },
            {
               "name": "the leg is bounded with white spots that covers the dead tissue of the leg.",
               "nameAr": "تتحزم قاعدة الساق بنمو أبيض يغطى الانسجة الميتة من الساق ينغمس في هذا النمو اجسام لونها بنى خفيف في حجم حبة الخردل",
               "CF": 0.03869203856456305
            }
         ]
      },
      {
         "name": " الذبول وعفن القاعدة\n",
         "link": "https://www.al-hakem.com/الذبول-وعفن-القاعدة-على-الطماطم/",
         "details": " الفطرالمسبب للذبول : Fusarium oxysporum F.sp.lycopersici",
         "symptoms": [
            {
               "name": "Sandy lands",
               "nameAr": "الأراضي رملية.",
               "CF": 0.5465114375697812
            },
            {
               "name": "Sagging for the lower leaves",
               "nameAr": "تهدل للأوراق السفلية",
               "CF": 0.7093617766929309
            },
            {
               "name": "yellowish leaves",
               "nameAr": "اصفرار الأوراق",
               "CF": 0.8454370442145466
            },
            {
               "name": "The leaves remain attached to the stem.",
               "nameAr": "تظل الأوراق معلقة على الساق.",
               "CF": 0.1
            },
            {
               "name": "Leaves dry",
               "nameAr": "جفاف الاوراق",
               "CF": 0.2
            },
            {
               "name": "Withering and drying of leaves occurs from the bottom to the top of the plant.",
               "nameAr": "يحدث ذبول الاوراق وجفافها من اسفل إلى أعلى النبات.",
               "CF": 0.5193551026141083
            },
            {
               "name": "When making a longitudinal cut in the stem or roots, dark brown stripes appear",
               "nameAr": "عند عمل شق طولى فى الساق أو الجذور يظهر خطوط لونها بنى داكن",
               "CF": 0.3956396379121305
            },
            {
               "name": "When making a longitudinal section of the root, the brown formation is observed at the center of the roo",
               "nameAr": "عند عمل قطاع طولي للجذر فيلاحظ التكون البنى في مركز الجذر",
               "CF": 0.13582741508842278
            }
         ]
      },
      {
         "name": "الذبول الفيوزايومي Fusarium wilt",
         "link": "https://www.al-hakem.com/الذبول-الفيوزايومي-على-الطماطم/",
         "details": "العامل المسبب : الفطر Fusarium Oxyspolum f. sp. Vasinfectum\nيُصيب البادرات ويقتلها بمجرد إنباتها بعد ظهورها فوق سطح التربة مما يؤدى إلى قلة عدد الشتلات الناتجة كما يصيب النباتات الكبيرة ويؤدى إلى قلة محصولها .",
         "symptoms": [
            {
               "name": "Yellowing and fading of the lower leaves of the plant",
               "nameAr": "إصفرار الأوراق السفلى للنبات وذبولها",
               "CF": 0.6015771208805073
            },
            {
               "name": "die branches",
               "nameAr": "موت الأفرع الصغيرة",
               "CF": 0.2
            },
            {
               "name": "brown color branches",
               "nameAr": "تحول الأفرع للون البنى",
               "CF": 0.2
            },
            {
               "name": "plant withers quickly, especially after the base of the stem rot",
               "nameAr": "يذبل النبات كلة بسرعة خاصة بعد تعفن قاعدة الساق",
               "CF": 0.49097410049715473
            },
            {
               "name": "The roots become soft and watery",
               "nameAr": "تصبح الجذور طرية ومائية وتظهر عليها عادة قروح لونها بنى مسود فى مواضع خروج الجذور الثانوية",
               "CF": 0.9669304645385947
            },
            {
               "name": "When large plants die before ripening, they will be yellow and fall to the ground",
               "nameAr": "عند موت النباتات الكبيرة قبل نضج الثمار فإنها تصفر وتتكرمش وتسقط على الأرض",
               "CF": 0.950639651638273
            },
            {
               "name": "the internal tissues are colored black",
               "nameAr": "عند نزع الأنسجة الخارجية للساق أو الجذور نلاحظ تلون الأنسجة الداخلية بلون مسود",
               "CF": 0.3656403393301745
            },
            {
               "name": "When making a longitudinal cut in the stem or roots, dark brown stripes appear",
               "nameAr": "عند عمل شق طولى فى الساق أو الجذور يظهر خطوط لونها بنى داكن",
               "CF": 0.814484728244465
            },
            {
               "name": "The infected root rot and discoloration in bluish or greenish color",
               "nameAr": "يتعفن الجذر المصاب ويتلون بلون مزرق أو مخضر",
               "CF": 0.6616053846706624
            },
            {
               "name": "Moisture in the soil",
               "nameAr": "رطوبة فى التربة",
               "CF": 0.1
            }
         ]
      }
   ],
   "بطاطا - potato": [
      {
         "name": "اليرقات البيضاء White Grubs\n",
         "link": "https://www.al-hakem.com/اليرقات-البيضاء-على-البطاطا/",
         "details": "الاسم العلمي : Phyllophaga spp. and other Scarabaeidae\nاليرقات البيضاء هي يرقات الخنافس الكبيرة نسبيا. قد يصل طول اليرقات البيضاء إلى 5 سم. أجسادهم قوية ومنحنية لها أرجل في الصدر.\nالحشرة تكمل دورة حياتها خلال سنه واحدة ، والحشرة الكامله تنشط في الصيف حيث تقوم بوضع البيض خلال هذه الفترة حول منطقة الجذور لنبات البطاطس .\n",
         "symptoms": [
            {
               "name": "Recessed holes on the surface of the tuber",
               "nameAr": "ثقوب غائرة على سطح الدرنة",
               "CF": 0.8811145856817809
            },
            {
               "name": "Economic damage occurs from the holes in the deep larvae made in the underground tubers",
               "nameAr": "يحدث التلف الاقتصادي من ثقوب اليرقات العميقة التي تصنع في الدرنات تحت الأرض",
               "CF": 0.831356985986292
            },
            {
               "name": "Extensive damage occurred when planting potatoes in pastures or pastoral fields",
               "nameAr": "حدث الضرر الشديد عند زراعة البطاطا في المراعي أو حقول الرعي السابقة",
               "CF": 0.3378936990772544
            }
         ]
      },
      {
         "name": "الذبول القمي الأورجواني purpule top wilt",
         "link": "https://www.al-hakem.com/الذبول-القمي-الأرجواني-على-البطاطا/",
         "details": "العامل المسبب ميكوبلازما : (Aster Yellows, Stolbur, Haywire)",
         "symptoms": [
            {
               "name": "stop growth",
               "nameAr": "توقف عن النمو",
               "CF": 0.8357637296027193
            },
            {
               "name": "The apical leaves are left curly and turn yellow or purple",
               "nameAr": "تترك الأوراق القميّة مجعدة وتصبح صفراء أو أرجوانية",
               "CF": 0.5660195110150391
            },
            {
               "name": "withering of the plant ",
               "nameAr": "تذبل النباتات",
               "CF": 0.9572196572199428
            },
            {
               "name": "Tubers from the affected plants do not germinate naturally and are small, flabby, deformed and without buds",
               "nameAr": "الدرنات من النباتات المصابة لا تنبت بشكل طبيعي وتكون صغيرة ، مترهلة ، مشوهة  وبدون براعم",
               "CF": 0.4465282232202562
            }
         ]
      },
      {
         "name": "نيماتودا تعقد الجذور Root Knot ",
         "link": "https://www.al-hakem.com/نيماتودا-تعقد-الجذور-على-البطاطا/",
         "details": "العامل المسبب : النيماتودا Meloidogyne spp",
         "symptoms": [
            {
               "name": "Abundant root and smell over the point of infection",
               "nameAr": " تكاثر جذور غزير ورائحة فوق نقطة العدوى",
               "CF": 0.08909681786049006
            },
            {
               "name": "Infestation with infected fungi roots",
               "nameAr": "اجتياح للجذور المصابة بالفطريات",
               "CF": 0.44919496516929946
            },
            {
               "name": "On the aerial parts of the plant, it appears stunting, targling and wilting",
               "nameAr": " يظهر على الأجزاء الهوائية للنبات التقزم والتبرقش والذبول.",
               "CF": 0.9458692569190955
            },
            {
               "name": "the crop is small",
               "nameAr": "المحصول الناتج يكون صغيرا بشكل عام وقد يكون غير قابل للتسويق",
               "CF": 0.884649696751569
            },
            {
               "name": "Tuber culosis is infected and shows deformities",
               "nameAr": "إصابة الدرنات بالعدوى وتظهر بها غورات أو تشوهات",
               "CF": 0.617495153610855
            },
            {
               "name": "plant death",
               "nameAr": "الموت المبكر للنبات",
               "CF": 0.06799172404200249
            }
         ]
      },
      {
         "name": "مرض تقرح الساق والقشرة السوداء Stem Canker and Black Scurf\n",
         "link": "https://www.al-hakem.com/تقرح-الساق-والقشرة-السوداء-على-البطاط/",
         "details": "العامل المسبب الفطر : Rhizoctonia solani\nيصيب هذا المرض اجزاء النبات الموجودة تحت سطح التربة، وقد يطلق عليه ايضا اسم تقرح الساق canker Stem او مرض الدرنات الهوائية، والفطر المسبب لهذا المرض له عوائل اخرى عديدة.",
         "symptoms": [
            {
               "name": "Long irregular ulcers of dark brown color appear in the stem of the plant below the soil surface",
               "nameAr": "يحدث في ساق النبات الموجودة أسفل سطح التربة تقرحات غير منتظمة الشكل طويلة ذات لون بني داكن",
               "CF": 0.9962645486104429
            },
            {
               "name": "die the stem of the plant below the soil surface",
               "nameAr": "موت ساق النبات الموجود أسفل سطح التربة",
               "CF": 0.1473472387001442
            },
            {
               "name": "die vegetabive sum",
               "nameAr": "موت الاجزاء الخضرية العلوية",
               "CF": 0.3020991420458685
            },
            {
               "name": "The plant becomes yellow in color and the leaves are blowing",
               "nameAr": "يصبح النبات اصفر اللون واوراقه متوردة",
               "CF": 0.7978343607809296
            },
            {
               "name": "The plant becomes yellow in color and the leaves are blurred",
               "nameAr": "يظهر على الدرنات اجساما حجرية سوداء اللون تلصق بسطح الدرنة التصاقا متينا",
               "CF": 0.4289605541413791
            },
            {
               "name": "Dry rot in tubers when stored",
               "nameAr": "حدوث عفن جاف بالدرنات عند تخزينها",
               "CF": 0.9475257684391936
            }
         ]
      },
      {
         "name": "عفن الطرف الجيلاتيني أو الهلامي Jell End Rot\n",
         "link": "https://www.al-hakem.com/عفن-الطرف-الجيلاتيني-أو-الهلامي-على-ال/",
         "details": "العامل المسبب : فيسيولوجي\nتحدث هذه الحالة نتيجة لزيادة تحول النشا إلي سكر عند قاعدة الدرنات ( خاصة الطويلة )",
         "symptoms": [
            {
               "name": "The base of the tubers is semi-transparent, but the skin remains healthy",
               "nameAr": "تكون قاعدة الدرنات شبه شفافة لكن يبقى الجلد سليماً",
               "CF": 0.5301854916856932
            },
            {
               "name": "The base of the tuber becomes gelatinous in appearance, but without any odor",
               "nameAr": "تصبح قاعدة الدرنة جيلاتينية المظهرولكن بلا أي رائحة",
               "CF": 0.04997536322414675
            },
            {
               "name": "Dry weather spread before harvest",
               "nameAr": "انتشار الجو الجاف قبيل الحصاد",
               "CF": 0.09728819589098181
            }
         ]
      },
      {
         "name": "ذبول البطاطا wlit",
         "link": "https://www.al-hakem.com/ذبول-البطاطا-أو-البطاطس/",
         "details": "العامل المسبب الفطر :  Verticillium and Fusarium\nتصاب البطاطس بالذبول الفيرتسيليومي أو الذبول الفيوزارمى أيضا .",
         "symptoms": [
            {
               "name": "Yellow leaves of the affected plants yellowish and extend upward",
               "nameAr": "اصفرار الأوراق السفلية للنباتات المصابة ثم يمتد لأعلى",
               "CF": 0.9299296039095305
            },
            {
               "name": "Leaves dry brown",
               "nameAr": "تجف الأوراق وتتحول إلى اللون البني",
               "CF": 0.1640743185190705
            },
            {
               "name": "By making a longitudinal incision in the stem of the plants just below or above the surface of the soil, you will see the coloring of the vascular bundles (brown wood) in the form of longitudinal brown lines.",
               "nameAr": "بعمل شق طولي في ساق النباتات أسفل أو أعلى سطح التربة بقليل يشاهد تلون الحزم الوعائية ( الخشب ) باللون البني في صورة خطوط بنية طولية",
               "CF": 0.5437180371784468
            },
            {
               "name": "When the tubers are cut transversally near their contact with the branch, a light colored, discrete brown circle is observed in the vascular bundles of the tuber.",
               "nameAr": "عند قطع الدرنات عرضياً بالقرب من اتصالها بالفرع يلاحظ وجود دائرة غير متصلة ملونة باللون البني الخفيف في الحزم الوعائية للدرنة",
               "CF": 0.32029986317766757
            }
         ]
      },
      {
         "name": "الجرب العادي Common Scab\n",
         "link": "https://www.al-hakem.com/الجرب-العادي-على-البطاطا/",
         "details": "العامل المسبب بكتريا : Streptomyces scabies\nالجرب هو عيب شائع في الدرنات في معظم مناطق زراعة البطاطس. يؤثر فقط على الجودة ، وليس العائد.\nو ينتقل هذا المرض عن طريق التقاوي المصابة. أو التربة الملوثة. أوالهواء. أو مياه الري.أو السماد العضوي الملوث.",
         "symptoms": [
            {
               "name": "Torn shell remains to cover the stain in the form of a thin, coarse film, taking the form of a star, and the edge of the stain is high",
               "nameAr": "خشونة في جلد الدرنة وقد تكون سطحية او مرتفعة على جلد الدرنة واحيانا تكون منخفضة عن سطح النسيج السليم",
               "CF": 0.2229845173185392
            },
            {
               "name": "The tubers lost their moisture, shrunk at storage and underweight",
               "nameAr": "فقد الدرنات للرطوبة وانكماشها عند التخزين ونقص وزنها",
               "CF": 0.34134083676652427
            },
            {
               "name": "Brown ulcers on the roots and stem.",
               "nameAr": "يتكون على الجذور والساق تقرحات لونها بني.",
               "CF": 0.3304887738388165
            },
            {
               "name": "Brown ulcers on the roots and stem.",
               "nameAr": "التربة جافة وقلوية ومرتفعة الحرارة",
               "CF": 0.12034588198098428
            }
         ]
      },
      {
         "name": "نقص النيتروجين (الاّزوت) Nitrogen Deficiency\n",
         "link": "https://www.al-hakem.com/نقص-النيتروجين-على-البطاطا/",
         "details": "",
         "symptoms": [
            {
               "name": "small leaves",
               "nameAr": "الأوراق أصغر من الأوراق الطبيعية",
               "CF": 0.8746727919388375
            },
            {
               "name": "yellowish-green leaves",
               "nameAr": " أوراق لونها أخضر باهت مصفر .",
               "CF": 0.029181702327315096
            },
            {
               "name": "The lower and more mature leaves are affected first",
               "nameAr": "تتأثر الأوراق المنخفضة والأكثر نضجا أولاً",
               "CF": 0.2696641724179145
            },
            {
               "name": "The leg is thin and solid.",
               "nameAr": "الساق تكون رقيقة وصلبة.",
               "CF": 0.36306702340091324
            },
            {
               "name": "Plants grow slowly",
               "nameAr": "النباتات تنمو ببطء.",
               "CF": 0.31412842494130744
            }
         ]
      },
      {
         "name": "نقص االفوسفور Phosphorus (P)\n",
         "link": "مزيد من المعلومات",
         "details": "",
         "symptoms": [
            {
               "name": "Plants grow slowly",
               "nameAr": "النباتات تنمو ببطء.",
               "CF": 0.2265734510033105
            },
            {
               "name": "The leg is thin, short and fragile",
               "nameAr": "الساق تكون رقيقة وقصيرة وهشة",
               "CF": 0.507811631139397
            },
            {
               "name": "The leaves appear purple in color, first on the underside, then later the old ones are affected first.",
               "nameAr": "يظهر على الأوراق لونًا أرجوانيًا ، أولًا على الجانب السفلي ، ثم في وقت لاحق تتأثر أولا الأوراق القديمة.",
               "CF": 0.46622108839883736
            }
         ]
      },
      {
         "name": "القلب الأسود Black Heart\n",
         "link": "https://www.al-hakem.com/القلب-الأسود-على-البطاطا/",
         "details": "تنتج هذه الظاهرة عن تخزين كميات كبيرة من الدرنات في الثلاجة أو عند سوء التهوية في النولات الأمر الذى ينتج عنه نقص الأكسجين وزيادة ثاني أكسيد الكربون نتيجة لارتفاع معدل التنفس ",
         "symptoms": [
            {
               "name": "A dead black space in the center of the tuber in the marrow tissue has no odor",
               "nameAr": "مساحة ميتة سوداء في وسط الدرنة في نسيج النخاع ليس لها رائحة",
               "CF": 0.7
            },
            {
               "name": "Soaking with water before harvest",
               "nameAr": "غمر التربة بالماء قبل الحصاد",
               "CF": 0.8935963691282107
            }
         ]
      },
      {
         "name": "القلب الأجوف Hollow Heart\n",
         "link": "https://www.al-hakem.com/القلب-الأجوف-على-البطاطا/",
         "details": "العامل المسبب: فيسيولوجي\nتزداد الإصابة بالقلب الأجوف في جميع الظروف التي تحفز النمو السريع للدرنات وتكوين درنات كبيرة الحجم وذلك عند زيادة التسميد العضوي قبل الزراعة وزيادة مسافات الزراعة أو زيادة نسبة الجور الغائبة وكذلك عند زيادة سرعة النمو الخضري يسبب ارتفاع درجة الحرارة أو زيادة الرطوبة الأرضية عند بداية تكوين الدرنات مما يزيد من سوء الحالة زيادة التسميد الآزوتى خاصة إذا ما جاءت هذه الظروف بعد فترة من الظروف القاسية التي يتوقف النمو خلالها.",
         "symptoms": [
            {
               "name": "One or more holes in the center of the tuber are generally surrounded by tissue from brown cork cells in the tissue of the bark",
               "nameAr": "حدوث فجوة أو عدة فجوات في وسط الدرنة تحاط بصفة عامة بنسيج من خلايا فلينية بنية اللون في نسيج اللحاء",
               "CF": 0.5679602870914829
            },
            {
               "name": "The infection occurs in large tubers",
               "nameAr": "تكثر الإصابة في الدرنات كبيرة الحجم",
               "CF": 0.5797550054425414
            },
            {
               "name": "Central cavity occurring if it begins to be formed in the early stages of tuber growth",
               "nameAr": "حدوث تجويف مركزي إذا بدأ تكوينه في المراحل المبكرة من نمو الدرنة",
               "CF": 0.18067965054486224
            },
            {
               "name": "A cavity occurs that is near one of the apical or basal tubercles if it begins to form late in the tuber growth",
               "nameAr": "حدوث تجويف يكون قريب من أحد طرفي الدرنة القمي أو القاعدي إذا بدأ تكوينه في مرحلة متأخرة من نمو الدرنة",
               "CF": 0.32334026624199474
            }
         ]
      },
      {
         "name": "العفن الأبيض White Mold\n",
         "link": "https://www.al-hakem.com/العفن-الأبيض-على-البطاطا/",
         "details": "العامل المسبب الفطر : Sclerotinia Sclerotiorum",
         "symptoms": [
            {
               "name": "Oval or sunken water spots on the stem of plants near their contact with the soil surface and extend to the top of the stem at first, then darken and later become brown and white in the middle, and they are annular",
               "nameAr": "بقع مائية غائرة بيضاوية أو مطاولة على ساق النباتات قرب اتصالها بسطح التربة وتمتد إلى أعلى الساق في بادئ الأمر ثم تغمق وتصبح في وقت لاحق بلون أسمر  وأبيض في الوسط, وتكون حلقية",
               "CF": 0.5532055134793225
            },
            {
               "name": "Stains appear covered with intense white cotton growth when moisture increases",
               "nameAr": "ظهور بقع تتغطى بنمو أبيض قطني كثيف عند اشتداد الرطوبة",
               "CF": 0.0379334044462567
            },
            {
               "name": "By making a longitudinal fissure, he sees the white mycelia growth inside the leg that becomes hollow on the inside",
               "nameAr": "بعمل شق طولي للساق يشاهد النمو الميسليومي الأبيض داخل الساق التي تصبح مجوفة من الداخل",
               "CF": 0.5989125679399285
            },
            {
               "name": "By making a longitudinal fissure, he sees the white mycelia growth inside the leg that becomes hollow on the inside",
               "nameAr": "تتكون أجسام صلبة سوداء تتراوح من 0.5 إلى 1.0 سم مكعب من الكتل متراصة في منطقة نخاع الساق",
               "CF": 0.8024344759723553
            },
            {
               "name": "The tubers become soft and have holes filled with fungal growth or even stone bodies",
               "nameAr": "تصبح الدرنات طرية وبها فجوات ممتلئة بالنمو الفطري أو حتى الأجسام الحجرية",
               "CF": 0.20969567564337577
            }
         ]
      },
      {
         "name": "الذبول البكتيري أو العفن البني Bacterial Wilt or  Brown Rot     \n",
         "link": "https://www.al-hakem.com/العفن-البني-على-البطاطا/",
         "details": "العامل المسبب بكتريا : Pseudomonas solanacearum\nمرض خطير جدا و ذو أثار تدميرية على محصول البطاطس في المناطق الدافئة والمدارية و نصف المدارية و بعض المناطق المعتدلة لكن يقل خطره في المناطق الباردة إلا أنه اكتشف حديثا وجود بعض سلالات تنتشر في المناطق الباردة .\nمرض العفن البني في البطاطس مدرج في الامراض الحجرية (أي انه يخضع للفحص الحجري للموافقة على الخروج لأي دولة مستوردة او الدخول اليها في صورة تقاوى) وهذا المرض غير مسموح بأي نسبة لتواجده في درنات البطاطس عند التصدير  .\nمزيد من المعلومات",
         "symptoms": [
            {
               "name": "Sudden wilt on seedlings leads to her death",
               "nameAr": "ذبول مفاجئ على البادرات يؤدي الى موتها",
               "CF": 0.15360983069820078
            },
            {
               "name": "Withering of large plants and leaves",
               "nameAr": "ذبول النباتات الكبيرة و الأوراق",
               "CF": 0.8701906707847987
            },
            {
               "name": "plant death",
               "nameAr": "موت النباتات ",
               "CF": 0.824094894522118
            },
            {
               "name": "A brown color appears in the vessels carrying the leg above the soil surface",
               "nameAr": "يظهر تلون بني في الأوعية الناقلة للساق فوق سطح التربة",
               "CF": 0.9107059823885334
            },
            {
               "name": "There are no external symptoms on the tubers",
               "nameAr": "لا تظهر أية أعراض خارجية على الدرنات",
               "CF": 0.4
            },
            {
               "name": "When making a cross section in the tuber, a sticky bacterial secretion of white and gray color is observed on the vascular bundle, which turns brown over time.",
               "nameAr": "عند عمل مقطع عرضي في الدرنة يلاحظ على دائرة الحزم الوعائية إفرازات بكتيرية لزجة غزيرة لونها ابيض رمادي تتحول إلى بني مع الوقت.",
               "CF": 0.3342467718583445
            },
            {
               "name": "When making a cross section in the tuber, a sticky bacterial secretion of white and gray color is observed on the vascular bundle, turning brown over time.",
               "nameAr": "تشقق في القشرة الخارجية للثمرة",
               "CF": 0.6874613510898229
            }
         ]
      },
      {
         "name": "الديدان السلكية أو فرقع اللوز Wireworms\n",
         "link": "https://www.al-hakem.com/فرقع-اللوز-على-البطاطا/",
         "details": "الاسم العلمي : أحد الأنواع التالية     Aeolus spp.    Anchastus spp.\n Melanotus spp.\nLimonius spp\n\nتهاجم اليرقات الجذور تحت سطح التربة. – فهي تهاجم القرص الجذرية والشعيرات الجذرية للنجيليات. و تهاجم الدرنات وتحفر فيها أنفاقا.",
         "symptoms": [
            {
               "name": "Tuber root rot with fungi and soil bacteria",
               "nameAr": "تعفن الجذور الدرنية بفطريات وبكتريا التربة.",
               "CF": 0.903639864857995
            },
            {
               "name": "generally there is Yellow color appears, wilted, and then dies",
               "nameAr": "في حالة الإصابة الشديدة يظهراصفرار عام ثم ذبول ثم الموت",
               "CF": 0.7902500790565513
            },
            {
               "name": "Tuber root rot with fungi and soil bacteria",
               "nameAr": "تظهر الإصابة في الحقول على شكل بقع متفرقة قابلة للانتشار",
               "CF": 0.06788812321551752
            },
            {
               "name": "The soil is heavy, wet clay and it is flooded with water in winter",
               "nameAr": "التربة طينية ثقيلة رطبة ويتم غمورها بالماء شتاء.",
               "CF": 0.29190321145218867
            }
         ]
      },
      {
         "name": "الجرب المسحوقي Powdery Scab\n",
         "link": "https://www.al-hakem.com/الجرب-المسحوقي-على-البطاطا/",
         "details": "العامل المسبب الفطر : Spongospora subterranean",
         "symptoms": [
            {
               "name": "Sandy soil",
               "nameAr": "التربة رملية",
               "CF": 0.6726347149305282
            },
            {
               "name": "The presence of small warts on the roots with a diameter ranging from 2-10 mm, which are light in color at first, then gradually darkened in color and later decomposed",
               "nameAr": "وجود ثاليل صغيرة على الجذور بقطر يتراوح بين 2-10 مم تكون ذات لون فاتح في البداية ثم بغمق لونها تدريجياً وتحلل فيما بعد",
               "CF": 0.1527801724749579
            },
            {
               "name": "Dryness and wilt of plants",
               "nameAr": "جفاف  وذبول النبات",
               "CF": 0.6880361571431834
            },
            {
               "name": "The temperature is low",
               "nameAr": "درجة الحرارة منخفضة",
               "CF": 0.319893036058842
            },
            {
               "name": "On the tubers there are small pimples (0.5-2 mm), which grow into light warts (0.5-1 cm), and in later stages the content of the warts becomes dark brown, then the warts shell explodes and spores are released in the form of a dark brown powder.",
               "nameAr": "يوجود على الدرنات بثرات صغيرة (0.5-2 مم) ,تنمو لتتحول إلى ثآليل فاتحة اللون (0.5-1 سم ) ,وفي المراحل المتأخرة يصبح محتوى الثآليل بني غامق ,بعدها تنفجر قشرة الثأليل وتتحرر منها أبواغ على شكل بودرة بنية مسودة",
               "CF": 0.43569972876630825
            },
            {
               "name": "Torn shell remains to cover the stain in the form of a thin, coarse film, taking the form of a star, and the edge of the stain is high",
               "nameAr": "بقاء لقشرة الممزقة مغطية للبقعة على شكل غشاء رقيق خشن , يأخذ شكل النجمة ,وتكون حافة البقعة مرتفعة .",
               "CF": 0.24453825481904046
            }
         ]
      }
   ],
   "رمان - pomegranate": [
      {
         "name": "مرض عفن القلب الأسود  Heart rot (Black heart) or Alternaria Fruit Rot ",
         "link": "https://www.al-hakem.com/مرض-عفن-القلب-الأسود-في-الرمان/",
         "details": "السبب : قد يتسبب عن الاصابة بالاسبرجللس Aspergillus أو الالترناريا Alternaria\nالسبب الرئيسي لحدوث الاصابة هي الظروف الجوية الرطبة خلال مرحلة الإزهار",
         "symptoms": [
            {
               "name": "The outer shell looks healthy",
               "nameAr": "تبدو القشرة الخارجية للثمار سليمة",
               "CF": 0.6064007625633545
            },
            {
               "name": "The internal content of the fruits is hollow",
               "nameAr": "المحتوى الداخلي للثمار يكون أجوف",
               "CF": 0.022607784963703192
            },
            {
               "name": "Inner mold is formed in brown to black color",
               "nameAr": "يتكون عفن داخلي ذو لون بني إلى أسود",
               "CF": 0.03565805798906252
            },
            {
               "name": "Fruits are generally lighter in weight, pale red in color, and sometimes reddish-brown.",
               "nameAr": "تكون الثمار عمومًا أخف وزناً وذات لون أحمر شاحب وفي بعض الأحيان تكون ذات لون بني محمر.",
               "CF": 0.014223221443316558
            }
         ]
      },
      {
         "name": "قافزات أوراق الكرز على الرمان Cherry Leafhopper\n",
         "link": "https://www.al-hakem.com/قافزات-أوراق-الرمان/",
         "details": "الاسم العلمي : Fieberiella florii\nتتغذى الحوريات والحشرات الكاملة من قافزات الأوراق leafhoppers على الجوانب السفلية من الأوراق ، باستخدام أجزاء الفم الثاقبة الماصة لامتصاص العصارة النباتية .",
         "symptoms": [
            {
               "name": "White burrs (small dots) on the top surface of the leaf",
               "nameAr": "نتوءات بيضاء (نقاط صغيرة) على السطح العلوي من الورقة",
               "CF": 0.6962257758668666
            },
            {
               "name": "White burrs (small dots) on the top surface of the leaf",
               "nameAr": "البقع البيضاء المنتقخة ، مما يتسبب في ظهور الأوراق بيضاء تقريبًا. مما يعطل التركيب الضوئي",
               "CF": 0.48238905524929676
            },
            {
               "name": "leaves fall",
               "nameAr": "تساقط الأوراق",
               "CF": 0.34349772653300825
            },
            {
               "name": "Black mold fungus growth",
               "nameAr": "نمو فطر العفن الأسود .",
               "CF": 0.26332037598501024
            }
         ]
      },
      {
         "name": "تعفن الثمارأو تقرح الساق بايليديلا Pilidiella Stem Canker and Fruit Rot\n",
         "link": "https://www.al-hakem.com/تعفن-الثمارأو-تقرح-ساق-الرمان/",
         "details": "العامل المسبب :الفطر Pilidiella granati\nتحدث عدوى الثمار من خلال الجروح و يمكن أن تنتشر العدوى عن طريق الثمارالمصابة إلى الثمار السليمة في الصناديق المعبأة.",
         "symptoms": [
            {
               "name": "Ulceration of the crown and leg",
               "nameAr": "تقرح التاج والساق",
               "CF": 0.6011370203282416
            },
            {
               "name": "break branches",
               "nameAr": "تكسر و موت الفرع",
               "CF": 0.9137559540484089
            },
            {
               "name": "fruits are embalmed",
               "nameAr": "تعفن الثمار الناضجة وتحنطها",
               "CF": 0.5847924240023542
            },
            {
               "name": "The seed housing becomes brown, and the membranes of the fruits and skin turn brown.",
               "nameAr": "تصبح مساكن البذور باللون البني وتتحول أغشية الثمار والقشرة إلى اللون البني.",
               "CF": 0.9716598158908648
            }
         ]
      }
   ],
   "موز - banana": [
      {
         "name": "نيماتودا تعقد الجذور\nRoot Knot\n",
         "link": "https://www.al-hakem.com/نيماتودا-تعقد-الجذور-على-الموز/",
         "details": "تصيب المجموعة الجذرية مسببة تعقد الجذور فتصفر الاوراق ويذبل النبات وتتعفن الجذور. تعالج الاصابة باستخدام المبيدات المتخصصة او بإعدام النباتات المصابة وتعقيم الجور.",
         "symptoms": [
            {
               "name": "spots or tumers on the roots",
               "nameAr": "وجود عقد أو أورام على جذور النباتات المصابة",
               "CF": 0.11980745661528103
            },
            {
               "name": "decreased in the crop",
               "nameAr": "قلة ونقص في كمية المحصول الناتج.",
               "CF": 0.6912678251413819
            },
            {
               "name": "yellowish leaves",
               "nameAr": "إصفرار الأوراق",
               "CF": 0.6699689033875497
            },
            {
               "name": "Molds may appear on the root system as a result of infection with nematodes.",
               "nameAr": "قد تظهر أعفان على المجموع الجذرى نتيجة الإصابة بالنيماتودا .",
               "CF": 0.479117284045801
            }
         ]
      },
      {
         "name": "مرض تورد قمة الموز \nBanana Bunchy Top",
         "link": "https://www.al-hakem.com/مرض-تورد-القمة-على-الموز/",
         "details": "مرض فيروسي خطير تنقله حشرة المن ويتسبب هذا المرض في تدمير مزارع الموز. يقاوم المرض بإعدام النباتات المصابة وتطهير الجور بالجير، كما يستخدم مبيد دايمثيوايت رشا على النباتات.",
         "symptoms": [
            {
               "name": "Dark green dots appear on the petioles of the leaves from the lower surface, and these dots appear clearly when exposing the affected leaves to sunlight",
               "nameAr": "ظهور نقط خضراء داكنة اللون على أعناق الأوراق من السطح السفلى ، وتظهر هذه النقط بوضوح عند تعريض الأوراق المصابة لأشعة الشمس",
               "CF": 0.8797485454969205
            },
            {
               "name": "Accumulated leaves of the affected plant on top of the plant in the form of a rose",
               "nameAr": "تراكم أوراق النبات المصاب على قمة النبات على شكل الوردة",
               "CF": 0.4972602422734518
            },
            {
               "name": "Leaf edges chlorosis",
               "nameAr": "إصفرار حواف الأوراق، يمتد إلى أنسجة الورقة بين العروق",
               "CF": 0.6984589420820531
            },
            {
               "name": "Leaf edges turns brown",
               "nameAr": "تتحول حواف الأوراق إلى اللون البني",
               "CF": 0.14876855248683263
            }
         ]
      },
      {
         "name": "منّ الموز\nBanana aphid",
         "link": "https://www.al-hakem.com/من-الموز/",
         "details": "وتأتى خطورة حشرات المن على الموز فى أنها تسبب أضرارا مباشرة وأخرى غير مباشرة على المحصول .",
         "symptoms": [
            {
               "name": "Distorted fruits",
               "nameAr": "تشوه الثمار وعدم صلاحيتها للتسويق",
               "CF": 0.9521940764506226
            },
            {
               "name": "Withering leaves",
               "nameAr": "ذبول الأوراق",
               "CF": 0.8066431595010156
            },
            {
               "name": "yellowish leaves",
               "nameAr": "إصفرار الأوراق",
               "CF": 0.3621084326667392
            },
            {
               "name": "Black mold fungus growth",
               "nameAr": "نمو فطر العفن الأسود .",
               "CF": 0.12986844836794764
            },
            {
               "name": "Molds may appear on the root system as a result of infection with nematodes.",
               "nameAr": "تواجد مستعمرات المن في تاج النبات عند عند المجموع الجذري أو بين أغماد الأوراق الخارجية",
               "CF": 0.1860906558507165
            }
         ]
      }
   ]
}


plants_Array = [
   "ملفوف - Cabbage",
   "زيتون - olive",
   "تفاح - Apple",
   "طماطم - tomatoes",
   "بطاطا - potato",
   "رمان - pomegranate",
   "موز - banana"
]

In [11]:
from constant import *
from db import *
from gui import *
import codecs

randomChoice = False

''' asking helper functions '''
file = codecs.open('samples/test.txt', 'w', 'utf-8')


def Ask_Choices_Question(question: str, *choices) -> str:
    return showAskAboutSymtomWindow(question, choices)

    '''
        function that ask question with list of choices to allow the user to choose from
    '''
    # // TODO make it in gui
    try:
        answer = input('{}\nchoices:\n{}\nanswer:'.format(
            question,
            '\n'.join(
                map(lambda choice: f"{choice[0] + 1}) {choice[1]}", enumerate(choices)))
        ))
        if not answer in choices:
            answer = choice(choices)
        raise IOError(f'wrong choice Try Again')
    except IOError as error:
        print(f'{B_COLORS.FAIL}ERROR: {error}{B_COLORS.ENDC}')


def Ask_Plant_Name() -> str:
    result = Ask_Choices_Question(
        'what is the plant to be diagnosed ?', *plants_Array)
    file.write(result + '\n')
    return result


def Ask_About_Symptom(question: str, symptom_Name, symptom_NameAr) -> float:
    if randomChoice:
        result = choice(list(CF_STRINGS.keys()))
        # result = 'مطلقاً'
        # result = 'لا'
        # result = 'لست متأكدا'
        # result = 'نعم ربما'
        # result = 'نعم بالتاكيد'
    else:
        result = Ask_Choices_Question(question, *CF_STRINGS.keys())
    file.write(symptom_NameAr + '\n')
    file.write(result + '\n')
    return CF_STRINGS[result]


In [13]:
''' Custom Defined Facts '''


class Disease(Fact):
    # Description: Defines the types of data describing the details of a Disease
    #
    # Properties:
    # -- name - Name of the disease
    # -- plant - Name of the plant that the disease belongs to
    # -- symptoms - The disease symptoms. It's dict object that map Name of the Symptom with symptoms data
    #             -- Symptom object have CF property that is the symptom contributes to the overall disease
    # -- CF - The total weight that the symptoms of the disease contribute. It's an addition of only the present symptoms.
    # -- state - The current state/stage of the disease in the system

    name = Field(str,mandatory=True)

    plant = Field(str, mandatory=True)
    symptoms = Field({str: dict}, mandatory=True)

    CF: Field(float)
    state: Field(DiseaseStates.values, DiseaseStates.INITIAL)


class UserInput(Fact):
    plant = Field(str, mandatory=True)
    symptoms = Field(
        Or({str: float}, object),
    )



In [15]:
''' helper function in defining disease stage'''


def Accumulate_Symptoms_CF(resultSymptoms: dict, symptom: dict):
    return {
        **resultSymptoms,
        symptom['name']: {
            **symptom
        }
    }


def Build_Disease_Fact(**disease: Disease) -> Disease:
    return Disease(
        **{
            **disease,
            'symptoms': reduce(
                Accumulate_Symptoms_CF,
                disease['symptoms'],
                {}
            ),
            'state': disease['state'] if 'state' in disease else DiseaseStates.INITIAL,
        }
    )



In [17]:
''' helper function in reading symptoms CF stage'''


def Accumulate_UserInput_Symptoms_CF(resultSymptomsCF: dict, symptom: dict) -> dict:
    # print('\nX\n', resultSymptomsCF, '\nW\n', symptom)
    symptom_Name = symptom['name']
    symptom_NameAr = symptom['nameAr']
    return {
        **resultSymptomsCF,
        symptom_Name: Ask_About_Symptom(f"what about {symptom_Name}?",symptom_Name,symptom_NameAr)
        if not symptom_Name in resultSymptomsCF
        else resultSymptomsCF[symptom_Name],
    }



In [ ]:
''' engines '''


class Input_Static_Data_Engine(KnowledgeEngine):

    @Rule(NOT(UserInput(plant=W())))
    def read_Plant_Name(self):
        # print('read plant')
        self.declare(UserInput(
            plant=Ask_Plant_Name(),
            symptoms={}
        ))


class Define_Plant_Diseases_Engine(KnowledgeEngine):

    @Rule(UserInput(plant=MATCH.plant), NOT(Fact(Diseases_Defined=True)))
    def define_Disease(self, plant):
        # print('define disease')
        for disease in Disease_Data[plant]:
            disease = {
                'plant': plant,
                **disease
            }
            self.declare(Build_Disease_Fact(**disease))
        self.declare(Fact(Diseases_Defined=True))


class Input_Symptoms_CF_Engine(KnowledgeEngine):

    @ Rule(
        AS.userInput << UserInput(
            plant=MATCH.userInputPlant,
            symptoms=MATCH.userInputSymptoms,
        ),
        AS.disease << Disease(
            plant=MATCH.diseasePlant,
            symptoms=MATCH.diseaseSymptoms,
            state=DiseaseStates.INITIAL,
        ),
        TEST(lambda diseasePlant, userInputPlant: diseasePlant == userInputPlant)
    )
    def readDiseaseSymptomsCF(
            self,
            userInput: UserInput,
            diseaseSymptoms: dict,
            disease: Disease,
            userInputSymptoms: dict,
    ):
        # initialize
        # print('read', userInput, repr(userInput), disease, repr(disease))
        Disease_Symptoms = diseaseSymptoms.values()
        UserInput_Symptoms_CF = unfreeze(userInputSymptoms) or {}

        # processing
        result_UserInput_Symptoms_CF = reduce(
            Accumulate_UserInput_Symptoms_CF,
            Disease_Symptoms,
            UserInput_Symptoms_CF
        )

        # print('result', result_UserInput_Symptoms_CF)

        # updating
        self.modify(userInput, symptoms=result_UserInput_Symptoms_CF)
        self.modify(disease, state=DiseaseStates.SY_CF_INPUTTED)


In [20]:
def Calc_Symptom_CF_With_UserInput(symptom: dict, userInputSymptomsCF: dict):
    symptomName = symptom['name']
    symptomCF = symptom['CF']
    return symptomCF * userInputSymptomsCF[symptomName]


def calcCF(cf1, cf2):
    print(cf1 * (1 - cf2))
    return cf1 * (1 - cf2)
    # TODO: check if it worked


class Diagnose_Disease_Engine(KnowledgeEngine):

    @Rule(
        AS.userInput << UserInput(
            plant=MATCH.userInputPlant,
            symptoms=MATCH.userInputSymptoms,
        ),
        AS.disease << Disease(
            plant=MATCH.diseasePlant,
            symptoms=MATCH.diseaseSymptoms,
            state=DiseaseStates.SY_CF_INPUTTED,
        ),
        TEST(lambda diseasePlant, userInputPlant: diseasePlant == userInputPlant),
        # may Test the cf Here or inside the RHS
    )
    def DiagnoseDisease(
            self,
            diseaseSymptoms: dict,
            disease: Disease,
            userInputSymptoms: dict,
    ):
        # pass
        # # initialize
        # print('read', disease, repr(disease))
        Disease_Symptoms = diseaseSymptoms.values()
        UserInput_Symptoms_CF = unfreeze(userInputSymptoms) or {}

        # # processing
        total_CF = 1 - reduce(
            lambda resultCF, symptom: calcCF(
                resultCF,
                Calc_Symptom_CF_With_UserInput(symptom, UserInput_Symptoms_CF)
            ),
            Disease_Symptoms,
            1
        )
        # print('result', result_UserInput_Symptoms_CF)

        # updating
        self.modify(disease, CF=total_CF, state=DiseaseStates.DIAGNOSED)

In [21]:
class MY_KE(
    Input_Static_Data_Engine,
    Define_Plant_Diseases_Engine,
    Input_Symptoms_CF_Engine,
    Diagnose_Disease_Engine,
    KnowledgeEngine,
):
    pass


engine = MY_KE()

engine.reset()
engine.run()
print(engine.facts)

TypeError: Ask_About_Symptom() missing 2 required positional arguments: 'symptom_Name' and 'symptom_NameAr'